In [1]:
# _*_ coding: utf-8 _*_
# @Time: 2021/12/09 16:05 
# @Author: yuyongsheng
# @Software: PyCharm
# @Description: 南方医院丙戊酸：1. 数据处理

# 原始数据预处理

## 导入程序包

In [2]:
# 导入程序包
import pymysql as MySQLDB
import pandas as pd
import numpy as np

import re
import sys
import os
project_path = os.getcwd()

## 导入预定义函数

In [3]:
# 字符串转换为时间格式
import datetime
def str_to_datetime(x):
    try:
        a = datetime.datetime.strptime(x, "%d/%m/%Y %H:%M:%S")
        return a
    except:
        return np.NaN

In [4]:
# 过滤文字
import numpy as np
def filter_string(df,feature):
    # 过滤文字!!!!!!!!!!!!!!!!!!!!!!!!!!!
    df=df[df[feature].str.contains('\d')]
    return df

In [5]:
# 过滤异常大值
def filter_exception_value(df,feature,median_value=1):
    # 过滤异常大值!!!!!!!!!!!!!!!!!!!!!!!!!!
    if not median_value:
        median_value=df[feature].median()
    df[feature]=df[feature].apply(lambda x: x if abs(float(x)) < (100 * abs(median_value)) else np.nan)
    df=df[df[feature].notnull()]
    return df

In [6]:
# 使用随机森林对缺失值进行插补
import pandas as pd
pd.set_option('mode.chained_assignment', None)
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
def missing_value_interpolation(df,missing_list=[]):
    df = df.reset_index(drop=True)
    # 提取存在缺失值的列名
    if not missing_list:
        for i in df.columns:
            if df[i].isnull().sum() > 0:
                missing_list.append(i)
    missing_list_copy = missing_list.copy()
    # 用该列未缺失的值训练随机森林，然后用训练好的rf预测缺失值
    for i in range(len(missing_list)):
        name=missing_list[0]
        df_missing = df[missing_list_copy]
        # 将其他列的缺失值用0表示。
        missing_list.remove(name)
        for j in missing_list:
            df_missing[j]=df_missing[j].astype('str').apply(lambda x: 0 if x=='nan' else x)
        df_missing_is = df_missing[df_missing[name].isnull()]
        df_missing_not = df_missing[df_missing[name].notnull()]
        y = df_missing_not[name]
        x = df_missing_not.drop([name],axis=1)
        # 列出参数列表
        tree_grid_parameter = {'n_estimators': list((10, 50, 100, 150, 200))}
        # 进行参数的搜索组合
        grid = GridSearchCV(RandomForestRegressor(),param_grid=tree_grid_parameter,cv=3)
        #rfr=RandomForestRegressor(random_state=0,n_estimators=100,n_jobs=-1)
        #根据已有数据去拟合随机森林模型
        grid.fit(x, y)
        rfr = RandomForestRegressor(n_estimators=grid.best_params_['n_estimators'])
        rfr.fit(x, y)
        #预测缺失值
        predict = rfr.predict(df_missing_is.drop([name],axis=1))
        #填补缺失值
        df.loc[df[name].isnull(),name] = predict
    return df

## 连接数据库

In [7]:
# 连接数据库
conn = MySQLDB.connect(host='192.168.0.37', port=3306, user='root', password='111111', db='nfyx_bingwusuan', charset='UTF8')
# conn = MySQLDB.connect(host='localhost', port=3306, user='root', password='123456', db='上海六院', charset='UTF8')
cursor = conn.cursor()
cursor.execute("Select version()")
for i in cursor:
    print(i)

('8.0.22',)


## 读取doctor_order数据库表

In [8]:
# 从df_doctor_order数据库读入数据
# 丙戊酸long_d_order不准，不要
try:
    sql = 'select patient_id,case_no,drug_name,amount,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc from doctor_order;'
    df_doctor_order = pd.read_sql(sql, conn)
except MySQLDB.err.ProgrammingError as e:
    print('surgical_record Error is ' + str(e))
    sys.exit()

In [9]:
print(df_doctor_order.shape)
print(df_doctor_order['patient_id'].nunique())

(35169, 10)
212


In [10]:
# 保存doctor_order用药记录
df_doctor_order.to_excel(project_path+'/data/pre_processed_raw_data/df_doctor_order.xlsx')

## 读取inp_record住院记录

In [11]:
# 从inp_record表读入数据
try:
    sql = 'select patient_id,case_no,adm_date,care_area,dis_date,age,cost from inp_record;'
    df_inp_record = pd.read_sql(sql, conn)
except MySQLDB.err.ProgrammingError as e:
    print('surgical_record Error is ' + str(e))
    sys.exit()

In [12]:
print(df_inp_record.shape)
print(df_inp_record['patient_id'].nunique())

(235, 7)
212


In [13]:
#用来判断某列是否有缺失值
df_inp_record.isnull().any() 

patient_id    False
case_no       False
adm_date      False
care_area     False
dis_date       True
age           False
cost          False
dtype: bool

In [14]:
df_inp_record[df_inp_record['dis_date'].isnull()]

,patient_id,case_no,adm_date,care_area,dis_date,age,cost
0,6873870,10001302,2020-08-15 17:12:30,JSKXLHLDY-心理二区护理单元江高,NaT,28,13004.95
2,6949939,10019847,2020-08-17 20:28:47,JSKXLHLDY-心理二区护理单元江高,NaT,27,9171.29
4,369329,10028911,2020-08-18 15:12:28,JSKXLHLDY-心理二区护理单元江高,NaT,33,9219.83
27,8043274,10914758,2020-11-18 17:38:49,JSKXLHLDY-心理二区护理单元江高,NaT,29,20882.11


In [15]:
# 查看重复数据
df_inp_record[df_inp_record.duplicated(subset=['patient_id','adm_date'],keep='first')]

,patient_id,case_no,adm_date,care_area,dis_date,age,cost


In [16]:
df_inp_record.to_excel(project_path+'/data/pre_processed_raw_data/df_inp_record.xlsx')

## 读取diagnostic_record

In [17]:
# 从medical_record表读入数据
try:
    sql = 'select patient_id,case_no,record_date,diagnostic_type,diagnostic_content from diagnostic_record;'
    df_diagnostic_record = pd.read_sql(sql, conn)
except MySQLDB.err.ProgrammingError as e:
    print('surgical_record Error is ' + str(e))
    sys.exit()

In [18]:
print(df_diagnostic_record.shape)
print(df_diagnostic_record['patient_id'].nunique())

(1818, 5)
353


In [19]:
#用来判断某列是否有缺失值
df_diagnostic_record.isnull().any() 

patient_id            False
case_no                True
record_date           False
diagnostic_type       False
diagnostic_content    False
dtype: bool

In [20]:
# 查看重复数据
df_diagnostic_record[df_diagnostic_record.duplicated(subset=['patient_id','case_no','record_date','diagnostic_content'],keep='first')]

,patient_id,case_no,record_date,diagnostic_type,diagnostic_content


In [21]:
writer=pd.ExcelWriter(project_path+'/data/pre_processed_raw_data/df_diagnostic_record.xlsx')
df_diagnostic_record.to_excel(writer)
writer.save()

## 读取test_result数据表

### test_record

In [22]:
# 从数据库读入test_record数据
try:
    sql = 'select test_record_id,patient_id,case_no,test_date,clinical_diagnosis,sample_type,sample_time from test_record;'
    df_test_record = pd.read_sql(sql, conn)
except MySQLDB.err.ProgrammingError as e:
    print('surgical_record Error is ' + str(e))
    sys.exit()

In [23]:
print(df_test_record.shape)
print(df_test_record['patient_id'].nunique())

(9669, 7)
358


In [24]:
df_test_record

,test_record_id,patient_id,case_no,test_date,clinical_diagnosis,sample_type,sample_time
0,800001202207||B001||1,7033812,None,2018-01-09 09:47:00,1:双相性情感障碍,血清,2018-01-09 08:05:35
1,800001202207||B002||1,7033812,None,2018-01-09 09:47:00,1:双相性情感障碍,血清,2018-01-09 08:05:35
2,800001202207||B005||1,7033812,None,2018-01-09 09:47:00,1:双相性情感障碍,血清,2018-01-09 08:05:35
3,800001202207||B006||1,7033812,None,2018-01-09 09:47:00,1:双相性情感障碍,血清,2018-01-09 08:05:35
4,800001202207||B007||1,7033812,None,2018-01-09 09:47:00,1:双相性情感障碍,血清,2018-01-09 08:05:35
...,...,...,...,...,...,...,...
9664,800013768402||B005||1,9290063,14107745,2021-11-09 10:44:00,脑器质性精神障碍,血清,NaT
9665,800013768402||B006||1,9290063,14107745,2021-11-09 10:44:00,脑器质性精神障碍,血清,NaT
9666,800013768402||B034||1,9290063,14107745,2021-11-09 10:44:00,脑器质性精神障碍,血清,NaT
9667,800013768403||A083||1,9290063,14107745,2021-11-09 08:42:00,脑器质性精神障碍,静脉血,NaT


In [25]:
#用来判断某列是否有缺失值
df_test_record.isnull().any() 

test_record_id        False
patient_id            False
case_no                True
test_date             False
clinical_diagnosis     True
sample_type           False
sample_time            True
dtype: bool

In [26]:
# 保存test_record
df_test_record.to_excel(project_path+'/data/pre_processed_raw_data/df_test_record.xlsx')

### test_result

In [27]:
# 从数据库读入test_result数据
try:
    sql = 'select test_record_id,project_name,test_result,result_unit,is_normal,refer_scope,synonym from test_result;'
    df_test_result = pd.read_sql(sql, conn)
except MySQLDB.err.ProgrammingError as e:
    print('surgical_record Error is ' + str(e))
    sys.exit()

In [28]:
print(df_test_result.shape)

(60001, 7)


In [29]:
#用来判断某列是否有缺失值
df_test_result.isnull().any() 

test_record_id    False
project_name      False
test_result       False
result_unit        True
is_normal          True
refer_scope        True
synonym            True
dtype: bool

In [30]:
# 查看重复数据
df_test_result[df_test_result.duplicated(subset=['test_record_id','project_name','test_result'],keep='first')]

,test_record_id,project_name,test_result,result_unit,is_normal,refer_scope,synonym
59467,800013465956||B042||1,免疫球蛋白,阴性(-),None,N,阴性(-),IgA_gd
59468,800013465956||B042||1,免疫球蛋白,阴性(-),None,N,阴性(-),IgM_gd


In [31]:
# 删除test_result数据
df_test_result=df_test_result.drop_duplicates(subset=['test_record_id','project_name','test_result'],keep='first')
df_test_result=df_test_result.reset_index(drop=True)

In [32]:
print(df_test_result.shape)

(59999, 7)


In [33]:
# 删除<>号
df_test_result['test_result']=df_test_result['test_result'].astype('str').apply(lambda x:x.replace('<',''))
df_test_result['test_result']=df_test_result['test_result'].astype('str').apply(lambda x:x.replace('>',''))

In [34]:
print(df_test_result.shape)

(59999, 7)


In [35]:
# 保存test_result
df_test_result.to_excel(project_path+'/data/pre_processed_raw_data/df_test_result.xlsx')

### test_record+test_result

In [36]:
df_test=pd.merge(df_test_record,df_test_result,on=['test_record_id'],how='inner')

In [37]:
print(df_test.shape)
print(df_test['patient_id'].nunique())

(59999, 13)
358


In [38]:
# 保存检测数据
df_test.to_excel(project_path+'/data/pre_processed_raw_data/df_test.xlsx')

## 体检数据

In [163]:
# 从数据库读入体检exam数据
try:
    sql = 'select patientId,examDateTime,normalInd,description from exam_record;'
    df_exam = pd.read_sql(sql, conn)
except MySQLDB.err.ProgrammingError as e:
    print('surgical_record Error is ' + str(e))
    sys.exit()

In [187]:
# 删除空数据
df_exam=df_exam[df_exam['description'].notnull()].reset_index(drop=True)

In [188]:
# 重命名
df_exam=df_exam.rename(columns={'patientId':'patient_id',
                       'examDateTime':'exam_time'})

In [189]:
print(df_exam.shape)
print(df_exam['patient_id'].nunique())

(652, 4)
188


In [190]:
df_exam

,patient_id,exam_time,normalInd,description
0,8242965,2020-09-01 00:33:40,阴性,两侧胸廓对称、所见骨质未见异常；两侧肺野透过度正常，未见异常密度增高影；两肺纹理略增粗、增多...
1,8242965,2020-09-01 00:33:40,阳性,双肾大小形态正常，轮廓清晰，包膜完整，实质回声均匀，集合系统回声正常，双侧输尿管不扩张。膀胱...
2,8242965,2020-09-01 00:33:40,阳性,双肾大小形态正常，轮廓清晰，包膜完整，实质回声均匀，集合系统回声正常，双侧输尿管不扩张。膀胱...
3,7006940,2020-09-01 00:33:40,阳性,主动脉扩张、迂曲、延长，主动脉结向左突出，见弧状钙化影；两侧肺野透光度正常，未见明确实变影；...
4,7006940,2020-09-01 00:33:40,阳性,肝脏形态大小正常，包膜完整光滑，肝内实质回声细密稍增强，肝内血管及胆管显示欠清晰，门静脉、胆...
...,...,...,...,...
647,7765586,2020-08-21 00:33:40,阳性,肝脏形态大小正常，包膜完整光滑，下缘角锐利，肝内实质回声均匀，肝内血管及胆管显示清晰，走行正...
648,7765586,2020-08-21 00:33:40,阳性,肝脏形态大小正常，包膜完整光滑，下缘角锐利，肝内实质回声均匀，肝内血管及胆管显示清晰，走行正...
649,369329,2020-08-20 00:33:40,阳性,两侧胸廓对称、所见骨质未见异常；两侧肺野透过度正常，右肺尖见体外异物影；两肺纹理略增粗、增多...
650,369329,2020-08-21 00:33:40,阳性,肝脏形态大小正常，包膜完整光滑，下缘角锐利，肝内实质回声均匀，肝内血管及胆管显示清晰，走行正...


In [191]:
df_exam.to_excel(project_path+'/data/pre_processed_raw_data/df_exam.xlsx')

## 提取人口学特征

### gender、race

In [39]:
# 从patient_info提取gender,race,birth_year
try:
    sql = 'select patient_id,gender,race,birth_year from patient_info;'
    df_patient_info = pd.read_sql(sql, conn)
except MySQLDB.err.ProgrammingError as e:
    print('surgical_record Error is ' + str(e))
    sys.exit()

In [40]:
print(df_patient_info.shape)
print(df_patient_info['patient_id'].nunique())

(358, 4)
358


In [41]:
df_patient_info

,patient_id,gender,race,birth_year
0,1007763,女,汉族,1982-07-14
1,1039505,女,汉族,1996-05-13
2,110166,男,汉族,2003-06-03
3,1112533,女,汉族,1996-08-21
4,1126688,男,汉族,1990-09-28
...,...,...,...,...
353,9165958,男,汉族,1999-12-25
354,9203877,男,汉族,2006-06-08
355,9261032,女,None,1999-06-21
356,9290063,男,汉族,1995-01-24


In [42]:
#用来判断某列是否有缺失值
df_patient_info.isnull().any() 

patient_id    False
gender        False
race           True
birth_year    False
dtype: bool

In [43]:
# 查看重复数据
df_patient_info[df_patient_info.duplicated(subset=['patient_id','gender','race'],keep='first')]

,patient_id,gender,race,birth_year


In [44]:
print(df_patient_info.shape)
print(df_patient_info['patient_id'].nunique())

(358, 4)
358


In [45]:
df_patient_info.to_excel(project_path+'/data/pre_processed_raw_data/人口学特征/df_patient_info.xlsx')

### age

In [46]:
df_age=df_inp_record[['patient_id','case_no','age']]

In [47]:
print(df_age.shape)
print(df_age['patient_id'].nunique())

(235, 3)
212


In [48]:
df_age

,patient_id,case_no,age
0,6873870,10001302,28
1,547307,10005574,25
2,6949939,10019847,27
3,7765586,10024628,16
4,369329,10028911,33
...,...,...,...
230,2583650,9818116,59
231,7411504,9835190,17
232,8334957,9890262,14
233,6865481,9910592,20


In [49]:
df_age.to_excel(project_path+'/data/pre_processed_raw_data/人口学特征/df_age.xlsx')

### height、weight

In [50]:
# 从patient_sign_record提取height、weight
try:
    sql = 'select patient_id,case_no,record_date,sign_type,record_content,record_unit from patient_sign_record;'
    df_patient_sign_record = pd.read_sql(sql, conn)
except MySQLDB.err.ProgrammingError as e:
    print('surgical_record Error is ' + str(e))
    sys.exit()

In [51]:
# 提取身高信息
df_height=df_patient_sign_record[df_patient_sign_record['sign_type'].str.contains('身高')]

In [52]:
print(df_height.shape)
print(df_height['patient_id'].nunique())

(246, 6)
211


In [53]:
df_height

,patient_id,case_no,record_date,sign_type,record_content,record_unit
0,6873870,10001302,2020-08-15 14:00:00,身高(cm),153,cm
79,547307,10005574,2020-08-16 14:00:00,身高(cm),162,cm
143,6949939,10019847,2020-08-17 14:00:00,身高(cm),160,cm
213,7765586,10024628,2020-08-18 14:00:00,身高(cm),168,cm
262,369329,10028911,2020-08-18 14:00:00,身高(cm),159,cm
...,...,...,...,...,...,...
18808,2583650,9818116,2020-07-27 14:00:00,身高(cm),158,cm
18845,7411504,9835190,2020-07-29 14:00:00,身高(cm),163,cm
18897,8334957,9890262,2020-08-04 14:00:00,身高(cm),166,cm
18958,6865481,9910592,2020-08-06 14:00:00,身高(cm),183,cm


In [54]:
# 过滤文字
df_height=filter_string(df_height,'record_content')

In [55]:
print(df_height.shape)
print(df_height['patient_id'].nunique())

(242, 6)
211


In [56]:
# 查看重复数据
df_height[df_height.duplicated(subset=['patient_id','sign_type'])]

,patient_id,case_no,record_date,sign_type,record_content,record_unit
2540,7808248,11200975,2020-12-16 14:00:00,身高(cm),155,cm
4842,8970838,13022476,2021-06-15 14:00:00,身高(cm),155,cm
6324,8881895,14103871,2021-10-13 14:00:00,身高(cm),168,cm
6425,8881895,14103871,2021-10-27 14:00:00,身高(cm),168,cm
6473,8881895,14103871,2021-11-03 14:00:00,身高(cm),168,cm
7114,7412218,4069083,2018-07-23 14:00:00,身高(cm),160,cm
7508,7475493,4191397,2018-08-13 06:00:00,身高(cm),173,cm
7545,7475493,4191397,2018-08-20 14:00:00,身高(cm),173,cm
7660,6865481,4255479,2018-08-14 14:00:00,身高(cm),182,cm
7786,6679951,4267891,2018-08-22 18:00:00,身高(cm),156,cm


In [57]:
# # 排序
# df_height=df_height.sort_values(by=['patient_id','record_date'],ascending=[True,True])
# # 删除重复数据
# df_height=df_height.drop_duplicates(subset=['patient_id','sign_type'],keep='last')

In [58]:
print(df_height.shape)
print(df_height['patient_id'].nunique())

(242, 6)
211


In [59]:
df_height.to_excel(project_path+'/data/pre_processed_raw_data/人口学特征/df_height.xlsx')

In [60]:
# 提取体重信息
df_weight=df_patient_sign_record[df_patient_sign_record['sign_type'].str.contains('体重')]

In [61]:
print(df_weight.shape)
print(df_weight['patient_id'].nunique())

(803, 6)
212


In [62]:
df_weight

,patient_id,case_no,record_date,sign_type,record_content,record_unit
11,6873870,10001302,2020-08-15 14:00:00,体重(kg),52,kg
30,6873870,10001302,2020-08-22 06:00:00,体重(kg),52,kg
60,6873870,10001302,2020-08-29 14:00:00,体重(kg),52,kg
90,547307,10005574,2020-08-16 14:00:00,体重(kg),52,kg
111,547307,10005574,2020-08-23 06:00:00,体重(kg),52,kg
...,...,...,...,...,...,...
19018,6865481,9910592,2020-08-20 18:00:00,体重(kg),80,kg
19019,6865481,9910592,2020-08-20 06:00:00,体重(kg),80,kg
19044,6865481,9910592,2020-08-27 14:00:00,体重(kg),80,kg
19057,8374053,9963165,2020-08-11 14:00:00,体重(kg),62,kg


In [63]:
# 过滤文字
df_weight=filter_string(df_weight,'record_content')

In [64]:
print(df_weight.shape)
print(df_weight['patient_id'].nunique())

(798, 6)
212


In [65]:
# 过滤异常值
df_weight=df_weight[df_weight['record_content']!=0]

In [66]:
print(df_weight.shape)
print(df_weight['patient_id'].nunique())

(798, 6)
212


In [67]:
# 查看重复数据
df_weight[df_weight.duplicated(subset=['patient_id','sign_type'])]

,patient_id,case_no,record_date,sign_type,record_content,record_unit
30,6873870,10001302,2020-08-22 06:00:00,体重(kg),52,kg
60,6873870,10001302,2020-08-29 14:00:00,体重(kg),52,kg
111,547307,10005574,2020-08-23 06:00:00,体重(kg),52,kg
140,547307,10005574,2020-08-30 14:00:00,体重(kg),52,kg
177,6949939,10019847,2020-08-24 14:00:00,体重(kg),45,kg
...,...,...,...,...,...,...
18990,6865481,9910592,2020-08-13 06:00:00,体重(kg),80,kg
19018,6865481,9910592,2020-08-20 18:00:00,体重(kg),80,kg
19019,6865481,9910592,2020-08-20 06:00:00,体重(kg),80,kg
19044,6865481,9910592,2020-08-27 14:00:00,体重(kg),80,kg


In [68]:
# # 排序
# df_weight=df_weight.sort_values(by=['patient_id','record_date'],ascending=[True,True])
# # 删除重复数据
# df_weight=df_weight.drop_duplicates(subset=['patient_id','sign_type'],keep='last')

In [69]:
print(df_weight.shape)
print(df_weight['patient_id'].nunique())

(798, 6)
212


In [70]:
df_weight.to_excel(project_path+'/data/pre_processed_raw_data/人口学特征/df_weight.xlsx')

### patient_info merge

In [71]:
# 取 patient_id和case_no
df_patient=df_height[['patient_id','case_no']]

In [72]:
# merge gender
df_patient_merge=pd.merge(df_patient,df_patient_info,on=['patient_id'],how='left')

In [73]:
# merge age
df_age_simple=df_age[['case_no','age']]
df_patient_merge=pd.merge(df_patient_merge,df_age_simple,on=['case_no'],how='left')

In [74]:
# merge height
df_height_simple=df_height[['case_no','record_content']]
df_height_simple=df_height_simple.rename(columns={'record_content':'身高'})
df_patient_merge=pd.merge(df_patient_merge,df_height_simple,on=['case_no'],how='left')

In [75]:
# merge weight
df_weight_simple=df_weight[['case_no','record_content']]
df_weight_simple=df_weight_simple.rename(columns={'record_content':'体重'})
df_patient_merge=pd.merge(df_patient_merge,df_weight_simple,on=['case_no'],how='left')

In [76]:
df_patient_merge

,patient_id,case_no,gender,race,birth_year,age,身高,体重
0,6873870,10001302,女,汉族,1992-11-06,28,153,52
1,6873870,10001302,女,汉族,1992-11-06,28,153,52
2,6873870,10001302,女,汉族,1992-11-06,28,153,52
3,547307,10005574,女,汉族,1995-05-13,25,162,52
4,547307,10005574,女,汉族,1995-05-13,25,162,52
...,...,...,...,...,...,...,...,...
958,6865481,9910592,男,汉族,2000-11-06,20,183,80
959,6865481,9910592,男,汉族,2000-11-06,20,183,80
960,6865481,9910592,男,汉族,2000-11-06,20,183,80
961,8374053,9963165,女,汉族,1997-09-15,23,160,62


In [77]:
print(df_patient_merge.shape)
print(df_patient_merge['patient_id'].nunique())

(963, 8)
211


In [78]:
df_patient_merge.to_excel(project_path+'/data/pre_processed_raw_data/人口学特征/df_patient_merge.xlsx')

# 纳排

## 纳入标准

### 提取丙戊酸用药

In [79]:
# 提取甲氨蝶呤的用药数据
# 注意药物缩写和药物音译名，wtf
df_VPA = df_doctor_order[df_doctor_order['drug_name'].str.contains('丙戊酸') & (~df_doctor_order['drug_name'].str.contains('药材科'))]

In [80]:
df_VPA

,patient_id,case_no,drug_name,amount,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc
55,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5g,2/日,2020-08-24 10:37:31,2020-09-04 11:38:00,停止
70,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5g,ONCE,2020-08-25 17:31:34,NaT,核实
105,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5g,2/日,2020-09-04 11:38:55,NaT,核实
150,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5g,1/日,2020-08-15 19:51:46,2020-08-24 10:37:37,停止
298,547307,10005574,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5g,ONCE,2020-08-16 16:41:46,NaT,核实
...,...,...,...,...,...,...,...,...,...,...
34998,6865481,9910592,丙戊酸钠糖浆[100ml:5g*1瓶],1.0,100ml*1瓶,10ml,2/日,2020-08-24 15:55:18,NaT,核实
35007,6865481,9910592,丙戊酸钠糖浆[100ml:5g*1瓶],3.0,100ml*1瓶,10ml,Tid,2020-08-26 11:38:49,NaT,核实
35061,6865481,9910592,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5g,2/日,2020-08-06 15:27:36,2020-08-11 11:13:42,停止
35088,8374053,9963165,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5g,2/日,2020-08-11 22:09:19,2020-08-18 08:48:00,停止


In [81]:
print(df_VPA.shape)
print(df_VPA['patient_id'].nunique())

(864, 10)
212


In [82]:
# 保存丙戊酸用药数据
df_VPA.to_excel(project_path+'/data/processed_data/df_VPA.xlsx')

### 双相情感障碍者

In [83]:
# 从diagnostic_record中提取双相情感障碍
df_BD=df_diagnostic_record[df_diagnostic_record['diagnostic_content'].str.contains('双相')]

In [84]:
df_BD['diagnostic_content'].unique()

array(['双相情感障碍，目前为伴有精神病性症状的躁狂发作', '双相情感障碍，目前为轻度或中度抑郁发作',
       '双相情感障碍目前为不伴精神病性症状躁狂发作', '双相情感障碍，目前为轻躁狂发作', '双相情感障碍，目前为中度抑郁发作',
       '双相情感障碍，目前为缓解状态', '双相情感障碍', '双相情感障碍，目前为混合性发作',
       '双相情感障碍目前为伴精神病的重度抑郁发作', '双相情感障碍目前为不伴精神病的重度抑郁发作', '非典型双相情感障碍',
       '双相情感障碍，目前为不伴有精神病性症状的重度抑郁发作', '双相情感障碍,快速循环型',
       '双相情感障碍，目前为躁狂发作，伴有精神病性症状', '双相情感障碍，目前为不伴有精神病性症状的躁狂发作', '双相性情感障碍',
       '双相情感障碍（恢复期）', '双相情感障碍,目前为中度抑郁发作', '双相情感障碍，伴有精神病性症状，目前为躁狂发作',
       '双相情感障碍不伴有精神病性症状的躁狂发作', '双相性情感障碍?', '双相情感障碍目前为伴精神病性症状的躁狂发作？',
       '双相情感障碍，目前为混合性发作？', '双相情感障碍，目前为重度抑郁发作', '双相情感障碍？',
       '双相情感障碍，目前为伴有精神病性症状的重度抑郁发作', '非典型双相情感障碍?',
       '双相情感障碍，目前为重度抑郁发作，伴有精神病性症'], dtype=object)

In [85]:
print(df_BD.shape)
print(df_BD['patient_id'].nunique())

(861, 5)
348


In [86]:
# 纳入双相情感障碍
df_VPA_BD=df_VPA[df_VPA['patient_id'].isin(df_BD['patient_id'].unique())]

In [87]:
print(df_VPA_BD.shape)
print(df_VPA_BD['patient_id'].nunique())

(854, 10)
208


In [88]:
df_VPA_BD_not=df_VPA[~ df_VPA['patient_id'].isin(df_BD['patient_id'].unique())]
df_VPA_BD_not['patient_id'].unique()

array(['7680010', '7506303', '7528046', '7512212'], dtype=object)

In [89]:
df_VPA_BD_not['patient_id'].unique()

array(['7680010', '7506303', '7528046', '7512212'], dtype=object)

### 18-65岁

In [90]:
df_age_adult=df_age[(df_age['age']>=18) & (df_age['age']<=65)]

In [91]:
df_age_adult

,patient_id,case_no,age
0,6873870,10001302,28
1,547307,10005574,25
2,6949939,10019847,27
4,369329,10028911,33
5,8242965,10130473,24
...,...,...,...
228,7299707,9781575,25
229,8325164,9812242,30
230,2583650,9818116,59
233,6865481,9910592,20


In [92]:
print(df_age_adult.shape)
print(df_age_adult['patient_id'].nunique())

(186, 3)
165


In [93]:
# 纳入年龄限制
df_VPA_age=df_VPA_BD[df_VPA_BD['patient_id'].isin(df_age_adult['patient_id'].unique())]

In [94]:
print(df_VPA_age.shape)
print(df_VPA_age['patient_id'].nunique())

(679, 10)
161


In [95]:
df_VPA_age.to_excel(project_path+'/data/processed_data/df_temp_VPA_age.xlsx')

### HAMD≥20，BRMS≥20

### 达到血药浓度稳态

#### 计算丙戊酸日剂量

In [96]:
df_VPA_dosage=df_VPA_age.copy()

# 清洗单次日剂量dosage，100ml--5g，1ml--0.05g
df_VPA_dosage['dosage']=df_VPA_dosage['dosage'].apply(lambda x: float(x.replace('ml', ''))*0.05 if 'ml' in x else x.replace('g', ''))

In [97]:
# 清洗频次
one=['1/午','ONCE','1/日','1/日(餐前)','1/早','1/晚','1/日']
two=['1/12小时','2/日']
three=['Tid','3/日（餐前）']
df_VPA_dosage['frequency']=df_VPA_dosage['frequency'].apply(lambda x: 1 if x in one else
                                                                2 if x in two else
                                                                3 if x in three else x)

In [98]:
# 计算日剂量
df_VPA_dosage['日剂量']=df_VPA_dosage['dosage'].astype('float') * df_VPA_dosage['frequency'].astype('float')

In [99]:
df_VPA_dosage

,patient_id,case_no,drug_name,amount,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc,日剂量
55,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,2,2020-08-24 10:37:31,2020-09-04 11:38:00,停止,1.0
70,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,1,2020-08-25 17:31:34,NaT,核实,0.5
105,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,2,2020-09-04 11:38:55,NaT,核实,1.0
150,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,0.5
298,547307,10005574,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,1,2020-08-16 16:41:46,NaT,核实,0.5
...,...,...,...,...,...,...,...,...,...,...,...
34998,6865481,9910592,丙戊酸钠糖浆[100ml:5g*1瓶],1.0,100ml*1瓶,0.5,2,2020-08-24 15:55:18,NaT,核实,1.0
35007,6865481,9910592,丙戊酸钠糖浆[100ml:5g*1瓶],3.0,100ml*1瓶,0.5,3,2020-08-26 11:38:49,NaT,核实,1.5
35061,6865481,9910592,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,2,2020-08-06 15:27:36,2020-08-11 11:13:42,停止,1.0
35088,8374053,9963165,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,2,2020-08-11 22:09:19,2020-08-18 08:48:00,停止,1.0


In [100]:
print(df_VPA_dosage.shape)
print(df_VPA_dosage['patient_id'].nunique())
print(df_VPA_dosage['case_no'].nunique())

(679, 11)
161
182


In [101]:
df_VPA_dosage.to_excel(project_path+'/data/processed_data/df_2.1.5.1_计算日剂量.xlsx')

#### 提取年月日

In [102]:
# 临时用药的结束时间=开始时间
aaa=df_VPA_dosage[df_VPA_dosage['end_datetime'].isnull()]
bbb=df_VPA_dosage[df_VPA_dosage['end_datetime'].notnull()]
aaa['end_datetime']=aaa['start_datetime']

In [103]:
aaa

,patient_id,case_no,drug_name,amount,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc,日剂量
70,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,1,2020-08-25 17:31:34,2020-08-25 17:31:34,核实,0.5
105,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,2,2020-09-04 11:38:55,2020-09-04 11:38:55,核实,1.0
298,547307,10005574,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,1,2020-08-16 16:41:46,2020-08-16 16:41:46,核实,0.5
355,6949939,10019847,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,2,2020-09-02 11:12:03,2020-09-02 11:12:03,核实,1.0
677,369329,10028911,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,1,2020-08-18 17:26:03,2020-08-18 17:26:03,核实,0.5
...,...,...,...,...,...,...,...,...,...,...,...
34957,6865481,9910592,丙戊酸钠糖浆[100ml:5g*1瓶],1.0,100ml*1瓶,0.5,2,2020-08-15 17:49:32,2020-08-15 17:49:32,核实,1.0
34979,6865481,9910592,丙戊酸钠糖浆[100ml:5g*1瓶],1.0,100ml*1瓶,0.5,2,2020-08-20 20:52:36,2020-08-20 20:52:36,核实,1.0
34998,6865481,9910592,丙戊酸钠糖浆[100ml:5g*1瓶],1.0,100ml*1瓶,0.5,2,2020-08-24 15:55:18,2020-08-24 15:55:18,核实,1.0
35007,6865481,9910592,丙戊酸钠糖浆[100ml:5g*1瓶],3.0,100ml*1瓶,0.5,3,2020-08-26 11:38:49,2020-08-26 11:38:49,核实,1.5


In [104]:
df_VPA_time=pd.concat([aaa,bbb],axis=0)
# 排序
df_VPA_time=df_VPA_time.sort_values(by=['patient_id','case_no','start_datetime'])

In [105]:
print(df_VPA_time.shape)
print(df_VPA_time['patient_id'].nunique())
print(df_VPA_time['case_no'].nunique())

(679, 11)
161
182


In [106]:
# 提取丙戊酸用药时间的年月日
df_VPA_time['start_date']=df_VPA_time['start_datetime'].apply(lambda x: str(x).split(' ')[0])
df_VPA_time['end_date']=df_VPA_time['end_datetime'].apply(lambda x: str(x).split(' ')[0])

In [107]:
df_VPA_time.to_excel(project_path+'/data/processed_data/df_2.1.5.2_提取年月日.xlsx')

#### 拆分长期医嘱和临时医嘱

In [108]:
# 同一剂量至少连续服用3天
# 打长期医嘱标签
df_VPA_time['long_d_order']=df_VPA_time['statusdesc'].apply(lambda x: 1 if re.match('停止',x) else 0)

In [109]:
# 长期医嘱和临时医嘱分开
drug_temp = df_VPA_time[df_VPA_time['long_d_order']==0]
drug_long = df_VPA_time[df_VPA_time['long_d_order']==1]

In [110]:
# 打临时时间标签
drug_temp['临时时间标签']=drug_temp['start_datetime'].apply(lambda x: 0 if x.hour <13 else 1)

In [111]:
print(drug_temp.shape)
print(drug_temp['patient_id'].nunique())
print(drug_temp['case_no'].nunique())

(394, 15)
154
174


In [112]:
# ======= 单独对‘临时医嘱’处理 =======
drug_temp_all = drug_temp.drop_duplicates(subset=['patient_id','start_date'],keep='first')
drug_temp_1 = drug_temp.drop_duplicates(subset=['patient_id','start_date','临时时间标签'],keep='first')
drug_temp_group = drug_temp_1.groupby(['patient_id','start_date'])['日剂量'].sum().reset_index()   # 同一天的临时医嘱相加，保证每天一条临时医嘱

# 拼接完整数据
drug_temp_result = pd.merge(drug_temp_all.drop(columns=['日剂量','临时时间标签']),drug_temp_group,on=['patient_id','start_date'],how='inner')

In [113]:
print(drug_temp_result.shape)
print(drug_temp_result['patient_id'].nunique())
print(drug_temp_result['case_no'].nunique())

(346, 14)
154
174


In [114]:
drug_temp_result

,patient_id,case_no,drug_name,amount,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc,start_date,end_date,long_d_order,日剂量
0,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,1,2021-03-29 19:14:29,2021-03-29 19:14:29,核实,2021-03-29,2021-03-29,0,0.5
1,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,2,2021-03-31 09:09:50,2021-03-31 09:09:50,核实,2021-03-31,2021-03-31,0,1.0
2,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,1,2021-04-07 09:10:18,2021-04-07 09:10:18,核实,2021-04-07,2021-04-07,0,0.5
3,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,1,2021-04-09 19:49:24,2021-04-09 19:49:24,核实,2021-04-09,2021-04-09,0,0.5
4,1173560,13397293,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,2,2021-07-21 16:21:42,2021-07-21 16:21:42,核实,2021-07-21,2021-07-21,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341,9290063,14107745,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,1,2021-09-22 18:47:53,2021-09-22 18:47:53,核实,2021-09-22,2021-09-22,0,0.5
342,9290063,14107745,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,1,2021-10-20 08:53:52,2021-10-20 08:53:52,核实,2021-10-20,2021-10-20,0,0.5
343,9290063,14107745,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,2,2021-11-08 09:01:08,2021-11-08 09:01:08,核实,2021-11-08,2021-11-08,0,1.0
344,9307105,14184132,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,1,2021-09-29 15:07:29,2021-09-29 15:07:29,核实,2021-09-29,2021-09-29,0,0.5


In [115]:
# 把‘临时医嘱’和‘长期医嘱’拼接
df_test_1 = pd.concat([drug_long,drug_temp_result],axis=0)
df_test_1 = df_test_1[drug_long.columns.tolist()]

# 拼接后重新排序
drug_vancomycin_4 = df_test_1.groupby(['case_no']).apply(
    lambda _df: _df.sort_values(by=['start_datetime'],ascending=1)).reset_index(drop=True)

In [116]:
drug_vancomycin_4

,patient_id,case_no,drug_name,amount,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc,日剂量,start_date,end_date,long_d_order
0,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,0.5,2020-08-15,2020-08-24,1
1,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,2,2020-08-24 10:37:31,2020-09-04 11:38:00,停止,1.0,2020-08-24,2020-09-04,1
2,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,1,2020-08-25 17:31:34,2020-08-25 17:31:34,核实,0.5,2020-08-25,2020-08-25,0
3,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,2,2020-09-04 11:38:55,2020-09-04 11:38:55,核实,1.0,2020-09-04,2020-09-04,0
4,547307,10005574,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,2,2020-08-16 16:41:46,2020-08-31 11:15:00,停止,1.0,2020-08-16,2020-08-31,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
626,6865481,9910592,丙戊酸钠糖浆[100ml:5g*1瓶],1.0,100ml*1瓶,0.5,2,2020-08-20 20:52:36,2020-08-20 20:52:36,核实,1.0,2020-08-20,2020-08-20,0
627,6865481,9910592,丙戊酸钠糖浆[100ml:5g*1瓶],1.0,100ml*1瓶,0.5,2,2020-08-24 15:55:18,2020-08-24 15:55:18,核实,1.0,2020-08-24,2020-08-24,0
628,6865481,9910592,丙戊酸钠糖浆[100ml:5g*1瓶],3.0,100ml*1瓶,0.5,3,2020-08-26 11:38:49,2020-08-26 11:38:49,核实,1.5,2020-08-26,2020-08-26,0
629,8374053,9963165,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,2,2020-08-11 22:09:19,2020-08-18 08:48:00,停止,1.0,2020-08-11,2020-08-18,1


In [117]:
drug_vancomycin_4.to_excel(project_path+'/data/processed_data/df_2.1.5.3_拆分临时医嘱.xlsx')

#### 分裂日期

In [118]:
# 将时间拆开
def explode1(df, col):
    dftem1 = pd.DataFrame()
    for i in range(df.shape[0]):
        dftem = df.iloc[i].to_frame().T
        if isinstance(df.iloc[i][col], list):
            if len(df.iloc[i][col]) > 0:
                for j in range(len(df.iloc[i][col])):
                    dftem[col + "_"] = df.iloc[i][col][j]
                    dftem1 = pd.concat([dftem1, dftem])
            else:
                dftem1 = pd.concat([dftem1, dftem])
        else:
            dftem1 = pd.concat([dftem1, dftem])
    del dftem1[col]
    dftem1.rename(columns={col + "_": col}, inplace=True)
    return dftem1

In [119]:
# 生成 分裂日期
drug_vancomycin_4['分裂日期'] = ''
for i in range(len(drug_vancomycin_4)):
    drug_vancomycin_4['分裂日期'][i] = list(pd.date_range(start=drug_vancomycin_4['start_date'][i], 
                                                      end=drug_vancomycin_4['end_date'][i]))

##　＝＝　把时间List拆开　＝＝    
drug_vancomycin_5 = explode1(drug_vancomycin_4,'分裂日期')
drug_vancomycin_5['分裂日期']=drug_vancomycin_5['分裂日期'].apply(lambda x: str(x).split(' ')[0])

drug_vancomycin_5['起始标签'] = drug_vancomycin_5.apply(lambda x: '起始' if x['分裂日期']==x['start_date'] else 
                                                                '结尾' if x['分裂日期']==x['end_date'] else 
                                                                '期间', axis=1)

In [120]:
drug_vancomycin_5

,patient_id,case_no,drug_name,amount,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc,日剂量,start_date,end_date,long_d_order,分裂日期,起始标签
0,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,0.5,2020-08-15,2020-08-24,1,2020-08-15,起始
0,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,0.5,2020-08-15,2020-08-24,1,2020-08-16,期间
0,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,0.5,2020-08-15,2020-08-24,1,2020-08-17,期间
0,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,0.5,2020-08-15,2020-08-24,1,2020-08-18,期间
0,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,0.5,2020-08-15,2020-08-24,1,2020-08-19,期间
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
629,8374053,9963165,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2020-08-11 22:09:19,2020-08-18 08:48:00,停止,1,2020-08-11,2020-08-18,1,2020-08-15,期间
629,8374053,9963165,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2020-08-11 22:09:19,2020-08-18 08:48:00,停止,1,2020-08-11,2020-08-18,1,2020-08-16,期间
629,8374053,9963165,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2020-08-11 22:09:19,2020-08-18 08:48:00,停止,1,2020-08-11,2020-08-18,1,2020-08-17,期间
629,8374053,9963165,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2020-08-11 22:09:19,2020-08-18 08:48:00,停止,1,2020-08-11,2020-08-18,1,2020-08-18,结尾


In [121]:
print(drug_vancomycin_5.shape)
print(drug_vancomycin_5['patient_id'].nunique())
print(drug_vancomycin_5['case_no'].nunique())

(3167, 16)
161
182


In [122]:
drug_vancomycin_5.to_excel(project_path+'/data/processed_data/df_2.1.5.4_分裂日期.xlsx')

#### 打时间标签

In [123]:
# 打日期标签
def flag_ttt(x):
    if (x['起始标签']!='结尾'):
        if x['start_datetime'].hour<13:
            return 0
        else:
            return 1
    else:
        if x['end_datetime'].hour<13:
            return 0
        else:
            return 1

In [124]:
# 上午是0，下午是1
# 起始和期间看 起始时间， 结尾看停止时间
drug_vancomycin_5['时间标签'] = drug_vancomycin_5.apply(lambda x: 0 if ((x['起始标签']!='结尾')and(x['start_datetime'].hour < 13)) else 1, axis=1)
drug_vancomycin_5['时间标签'] = drug_vancomycin_5.apply(flag_ttt,axis=1)

In [125]:
print(drug_vancomycin_5.shape)
print(drug_vancomycin_5['patient_id'].nunique())
print(drug_vancomycin_5['case_no'].nunique())

(3167, 17)
161
182


In [126]:
drug_vancomycin_5.to_excel(project_path+'/data/processed_data/df_2.1.5.5_打时间标签.xlsx')

#### 调整日剂量

In [530]:
temp_l = []

for i in drug_vancomycin_5.case_no.unique():
    temp = drug_vancomycin_5[drug_vancomycin_5['case_no']==i].reset_index(drop=True)
    
#     a = 1
#     print('===={}===='.format(i))
    for j in temp.分裂日期.unique():
#         print(a,j)
#         a+=1
        temp_2 = temp[temp['分裂日期']==j]
        if 0 in temp_2['long_d_order'].unique():
            temp_long = temp_2[temp_2['long_d_order']==1].tail(1).reset_index(drop=True)
            temp_st = temp_2[temp_2['long_d_order']==0]
            temp_st['日剂量'] = temp_st['日剂量'].sum()
            temp_st = temp_st.tail(1).reset_index(drop=True)
#             print(temp_long.shape[0],temp_st.shape[0])
            # 同一天，既有长期医嘱又有短期医嘱
            if len(temp_long)>0:
#                 print('long>0')
                flag = temp_long.起始标签.unique()[0]
                if flag == '期间':
                    temp_long.loc[0,'日剂量'] = temp_long.loc[0,'日剂量'] + temp_st.loc[0,'日剂量']                           
                    temp_l.append(temp_long)
                else:
                    temp_l.append(temp_st)
            # 该天全是短期医嘱
            else:
#                 print(temp_st)
#                 print('***')
                temp_l.append(temp_st)
        # 该天全是长期医嘱
        else:
            temp_long = temp_2.tail(1).reset_index(drop=True)
            flag_l = temp_2.起始标签.unique().tolist()
            flag_time_l = temp_2.时间标签.unique().tolist()
            if ('结尾' in flag_l and '起始' in flag_l) or len(flag_time_l)>1:
                # 按剂量去重（防止上一次结束，与下一次开始是两种情况）
                temp_long.loc[0,'日剂量'] = temp_2.drop_duplicates(subset=['日剂量'],keep='last').日剂量.mean()
                temp_l.append(temp_long)
            else:
                flag = temp_long.loc[0,'起始标签']
                flag_time = temp_long.loc[0,'时间标签']
                # 开头：大于13点取半天，  结尾：小于13点取半天
                if flag=='起始' and flag_time==1:
                    temp_long.loc[0,'日剂量'] = temp_long.loc[0,'日剂量'] / 2
                if flag=='结尾' and flag_time==0:
                    temp_long.loc[0,'日剂量'] = temp_long.loc[0,'日剂量'] / 2

                temp_l.append(temp_long)
        
df_VPA_modification = pd.concat(temp_l,axis=0).reset_index(drop=True)
df_VPA_modification['日剂量']=df_VPA_modification['日剂量'].apply(lambda x: round(x,2))
del temp_l

In [531]:
print(df_VPA_modification.shape)
print(df_VPA_modification['patient_id'].nunique())
print(df_VPA_modification['case_no'].nunique())

(2677, 17)
161
182


In [129]:
df_VPA_modification

,patient_id,case_no,drug_name,amount,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc,日剂量,start_date,end_date,long_d_order,分裂日期,起始标签,时间标签
0,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,0.25,2020-08-15,2020-08-24,1,2020-08-15,起始,1
1,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,0.50,2020-08-15,2020-08-24,1,2020-08-16,期间,1
2,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,0.50,2020-08-15,2020-08-24,1,2020-08-17,期间,1
3,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,0.50,2020-08-15,2020-08-24,1,2020-08-18,期间,1
4,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,0.50,2020-08-15,2020-08-24,1,2020-08-19,期间,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2672,8374053,9963165,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2020-08-11 22:09:19,2020-08-18 08:48:00,停止,1.00,2020-08-11,2020-08-18,1,2020-08-14,期间,1
2673,8374053,9963165,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2020-08-11 22:09:19,2020-08-18 08:48:00,停止,1.00,2020-08-11,2020-08-18,1,2020-08-15,期间,1
2674,8374053,9963165,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2020-08-11 22:09:19,2020-08-18 08:48:00,停止,1.00,2020-08-11,2020-08-18,1,2020-08-16,期间,1
2675,8374053,9963165,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2020-08-11 22:09:19,2020-08-18 08:48:00,停止,2.00,2020-08-11,2020-08-18,1,2020-08-17,期间,1


In [442]:
df_VPA_modification.to_excel(project_path+'/data/processed_data/df_2.1.5.6_调整日剂量.xlsx')

#### tdm稳态筛选

In [512]:
df_VPA_steady=drug_vancomycin_6.copy()
# 到稳态血药浓度（同一剂量至少连续服用3天)
temp_list=[]
for i in df_VPA_steady['case_no'].unique():
    print(i)
    temp=df_VPA_steady[df_VPA_steady['case_no']==i]
    temp=temp.reset_index(drop=True)
    j=0
    while j < temp.shape[0]:
#         print(j)
        # 连续服药3天，tdm达到稳态。1.剂量连续3天；2. 时间连续3天
        dosage_judge=temp.loc[j,'日剂量']    
        # 1. 剂量判断
        break_bool=False
        for k in range(j+1,temp.shape[0]):
            differ = (pd.to_datetime(temp.loc[k,'分裂日期'])-pd.to_datetime(temp.loc[k-1,'分裂日期'])).days
            if dosage_judge != temp.loc[k,'日剂量'] or differ != 1:
                # 判断是否中断
                break_bool=True
                break
        # 是否是最后一天
        if k != temp.shape[0]-1:
            # 是否是稳态
            if k-1-j>=2:
                temp_list.append(temp.loc[j:k-1,])
            j=k
            continue
        else:
            # 最后一天是否中断
            if break_bool:
                # 是否是稳态
                if k-1-j>=2:
                    temp_list.append(temp.loc[j:k-1,])
            else:
                if k-j >=2:
                    temp_list.append(temp.loc[j:k,])
            j=k+1

10001302
10005574
10019847
10028911
10130473
10146112
10171606
10200584
10297961
10343367
10357709
10368285
10389652
10582637
10588084
10674663
10692337
10830666
10877290
10914758
10941636
10989230
11022153
11069250
11200975
11238946
11291598
11351405
11547061
11747590
11787937
12069121
12070485
12210690
12297734
12313528
12325442
12631919
12693940
12696147
12782405
12855725
13022476
13063464
13150462
13176502
13317922
13397293
13491055
13561501
13817948
13951019
13997754
14040566
14103871
14107745
14135942
14184132
4028318
4045163
4069083
4103620
4112018
4133350
4151975
4191397
4216082
4236927
4255479
4267891
4283749
4304073
4364618
4366097
4446056
4446680
4536465
4553217
4560293
4569640
4592169
4707313
4761372
4821017
4842956
4860902
4891032
4918748
4922567
4926506
5125578
5180331
5278319
5302878
5386601
5407981
5436932
5544859
5612768
5634908
5653155
5679938
5682081
5683098
5696642
5708206
5722142
5779804
5968455
6000741
6085020
6145716
6158479
6242952
6268920
6324940
6338386
636182

In [513]:
# list转换为DataFrame
df_VPA_steady = pd.concat(temp_list,axis=0).reset_index(drop=True)
del temp_list

In [514]:
print(df_VPA_steady.shape)
print(df_VPA_steady['patient_id'].nunique())
print(df_VPA_steady['case_no'].nunique())

(1983, 18)
121
131


In [515]:
df_VPA_steady

,patient_id,case_no,drug_name,amount,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc,日剂量,start_date,end_date,long_d_order,分裂日期,起始标签,时间标签,连续标签
0,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,0.5,2020-08-15,2020-08-24,1,2020-08-16,期间,1,1
1,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,0.5,2020-08-15,2020-08-24,1,2020-08-17,期间,1,2
2,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,0.5,2020-08-15,2020-08-24,1,2020-08-18,期间,1,3
3,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,0.5,2020-08-15,2020-08-24,1,2020-08-19,期间,1,4
4,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,0.5,2020-08-15,2020-08-24,1,2020-08-20,期间,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1978,8374053,9963165,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2020-08-11 22:09:19,2020-08-18 08:48:00,停止,1.0,2020-08-11,2020-08-18,1,2020-08-12,期间,1,1
1979,8374053,9963165,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2020-08-11 22:09:19,2020-08-18 08:48:00,停止,1.0,2020-08-11,2020-08-18,1,2020-08-13,期间,1,2
1980,8374053,9963165,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2020-08-11 22:09:19,2020-08-18 08:48:00,停止,1.0,2020-08-11,2020-08-18,1,2020-08-14,期间,1,3
1981,8374053,9963165,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2020-08-11 22:09:19,2020-08-18 08:48:00,停止,1.0,2020-08-11,2020-08-18,1,2020-08-15,期间,1,4


In [516]:
df_VPA_steady.to_excel(project_path+'/data/processed_data/df_2.1.5.7_tdm稳态筛选.xlsx')

### 无躯体疾病

In [487]:
# 无严重的或不稳定的躯体疾病，且经检查表明躯体健康（包括体检查、病史、心电图和血生化、血液学及尿检查）

#### 无体检健康异常

In [488]:
df_VPA_exam=df_VPA_steady.copy()
# 删除体检异常
for i in range(df_exam.shape[0]):
    description=df_exam.loc[i,'description']
    description_list=re.split('，|。',description)
    for k in description_list:
        if re.match('[^无否未见]*异常',k):
            print(i)
            df_VPA_exam=df_VPA_exam[df_VPA_exam['patient_id']!=df_exam.loc[i,'patient_id']]
            break

In [489]:
print(df_VPA_exam.shape)
print(df_VPA_exam['patient_id'].nunique())
print(df_VPA_exam['case_no'].nunique())

(1983, 18)
121
131


#### 病史

In [490]:
df_diagnostic_record.columns

Index(['patient_id', 'case_no', 'record_date', 'diagnostic_type',
       'diagnostic_content'],
      dtype='object')

In [491]:
# 提取肝炎
df_HCA=df_diagnostic_record[df_diagnostic_record['diagnostic_content'].str.contains('肝炎')]

In [492]:
df_HCA.shape

(11, 5)

In [493]:
# 剔除肝炎患者
df_VPA_exam=df_VPA_exam[~ df_VPA_exam['patient_id'].isin(df_HCA.patient_id.unique())]

In [494]:
print(df_VPA_exam.shape)
print(df_VPA_exam['patient_id'].nunique())
print(df_VPA_exam['case_no'].nunique())

(1940, 18)
118
128


In [495]:
# 提取肾病，不包括肾结石
df_NEP=df_diagnostic_record[(df_diagnostic_record['diagnostic_content'].str.contains('肾'))&
                           (~ df_diagnostic_record['diagnostic_content'].str.contains('肾结石'))]

In [496]:
df_NEP

,patient_id,case_no,record_date,diagnostic_type,diagnostic_content
555,8881895,14103871,2021-09-30 10:31:17,出院诊断,慢性肾功能不全
558,8881895,14103871,2021-09-30 10:31:17,出院诊断,肾上腺肿物
566,8881895,14103871,2021-10-19 16:09:54,出院诊断,慢性肾脏病3期
568,8881895,14103871,2021-09-22 15:38:17,初步诊断,肾上腺肿物
572,8881895,14103871,2021-09-22 15:38:17,初步诊断,慢性肾功能不全
709,6591495,4151975,2018-08-13 12:13:48,出院诊断,右肾囊肿
714,6591495,4151975,2018-08-13 13:27:42,最后诊断,右肾囊肿
1072,7656474,5436932,2019-01-25 15:02:03,最后诊断,肾囊肿
1077,7656474,5436932,2019-01-23 21:06:56,出院诊断,肾囊肿
1119,5265125,5682081,2019-03-21 10:35:59,出院诊断,肾囊肿


In [497]:
# 剔除肾病患者
df_VPA_exam=df_VPA_exam[~ df_VPA_exam['patient_id'].isin(df_NEP.patient_id.unique())]

In [498]:
print(df_VPA_exam.shape)
print(df_VPA_exam['patient_id'].nunique())
print(df_VPA_exam['case_no'].nunique())

(1883, 18)
114
124


In [499]:
# 提取高血压
df_EH=df_diagnostic_record[df_diagnostic_record['diagnostic_content'].str.contains('高血压')]

In [500]:
df_EH

,patient_id,case_no,record_date,diagnostic_type,diagnostic_content
216,2930481,11022153,2020-12-02 14:44:52,初步诊断,高血压2级
218,2930481,11022153,2020-12-18 08:28:40,出院诊断,高血压2级
221,2930481,11022153,2020-12-19 02:12:10,出院诊断,高血压3级
712,6591495,4151975,2018-08-13 13:27:42,最后诊断,高血压病
716,6591495,4151975,2018-08-01 16:41:39,初步诊断,高血压病
718,6591495,4151975,2018-08-13 12:01:49,出院诊断,高血压病
873,2583650,None,2018-10-19 11:56:33,门诊诊断,高血压3级
1057,2583650,5407981,2019-01-17 12:35:55,出院诊断,高血压病
1064,2583650,5407981,2019-01-10 15:28:12,初步诊断,高血压病
1067,2583650,5407981,2019-01-17 12:21:54,最后诊断,高血压病


In [501]:
# 剔除高血压患者
df_VPA_exam=df_VPA_exam[~ df_VPA_exam['patient_id'].isin(df_EH.patient_id.unique())]

In [502]:
print(df_VPA_exam.shape)
print(df_VPA_exam['patient_id'].nunique())
print(df_VPA_exam['case_no'].nunique())

(1873, 18)
113
122


In [503]:
# 提取糖尿病
df_DM=df_diagnostic_record[df_diagnostic_record['diagnostic_content'].str.contains('糖尿病')]

In [504]:
df_DM

,patient_id,case_no,record_date,diagnostic_type,diagnostic_content
34,7006940,10146112,2020-09-10 10:14:26,出院诊断,2型糖尿病
35,7006940,10146112,2020-09-10 10:14:26,出院诊断,2型糖尿病周围神经病变
44,7006940,10146112,2020-08-31 15:23:24,初步诊断,2型糖尿病
46,7006940,10146112,2020-08-31 15:23:56,初步诊断,2型糖尿病周围神经病变
516,7651365,13817948,2021-08-26 19:40:05,初步诊断,2型糖尿病
548,8881895,14103871,2021-09-23 09:08:15,初步诊断,2型糖尿病
552,8881895,14103871,2021-09-30 10:31:17,出院诊断,2型糖尿病
567,8881895,14103871,2021-10-20 08:26:51,出院诊断,2型糖尿病周围神经病变
775,5878393,4366097,2018-09-13 11:39:26,最后诊断,2型糖尿病
778,5878393,4366097,2018-08-28 15:58:34,初步诊断,糖尿病


In [505]:
# 剔除高血压患者
df_VPA_exam=df_VPA_exam[~ df_VPA_exam['patient_id'].isin(df_DM.patient_id.unique())]

In [506]:
print(df_VPA_exam.shape)
print(df_VPA_exam['patient_id'].nunique())
print(df_VPA_exam['case_no'].nunique())

(1861, 18)
112
121


In [508]:
# 保存纳入标准数据
df_VPA_exam.to_excel(project_path+'/data/processed_data/df_2.1.6_无躯体疾病.xlsx')

#### 血液检测

#### 尿检

## 排除标准

### tdm稳态前给药方案发生改变

### 筛选前4周内接受过抗抑郁治疗或电休克治疗

### 妊娠或哺乳期妇女

### 排除使用肝药酶诱导剂

In [369]:
# 肝药酶诱导剂（如卡马西平、苯妥英钠、苯巴比妥等）

### 排除严重肝、肾疾病

### 排除数据严重缺失

In [370]:
# 数据严重缺失（如患者ID、主要研究结局等

### 排除物质滥用者

### 排除脑器质疾病或躯体疾病所致的情感障碍

In [371]:
# 提取脑器质疾病brain organic mental disorders
df_bomd=df_diagnostic_record[(df_diagnostic_record['diagnostic_content'].str.contains('器质'))&
                            df_diagnostic_record['diagnostic_content'].str.contains('精神')]

In [372]:
df_bomd

,patient_id,case_no,record_date,diagnostic_type,diagnostic_content
551,8881895,14103871,2021-09-30 10:31:17,出院诊断,脑器质性精神障碍
575,9290063,14107745,2021-10-20 09:00:47,初步诊断,脑器质性精神障碍
577,9290063,14107745,2021-11-09 08:37:44,出院诊断,脑器质性精神障碍
783,7506303,4389270,2018-09-14 08:16:50,最后诊断,器质性精神障碍
784,7506303,4389270,2018-09-14 08:17:28,出院诊断,器质性精神障碍


In [509]:
# 剔除脑器质疾病
df_VPA_exclusion=df_VPA_exam[~ df_VPA_exam['patient_id'].isin(df_bomd.patient_id.unique())]

In [510]:
print(df_VPA_exclusion.shape)
print(df_VPA_exclusion['patient_id'].nunique())
print(df_VPA_exclusion['case_no'].nunique())

(1861, 18)
112
121


In [511]:
df_VPA_exclusion.to_excel(project_path+'/data/processed_data/df_2_纳排数据.xlsx')

# 合并人口学特征

In [517]:
# 提取人口学合并特征
df_popu=df_patient_merge[['patient_id','gender','race','age','身高','体重']]
# 删除重复数据
df_popu=df_popu.drop_duplicates(['patient_id','gender','race'])

In [518]:
print(df_popu.shape)
print(df_popu['patient_id'].nunique())

(211, 6)
211


## 计算BMI

In [519]:
BMI_list=[round(x*10000/y/y,2) for (x,y) in zip(df_popu['体重'].astype('float').tolist(),df_popu['身高'].astype('float').tolist())]

In [520]:
print(len(BMI_list))
print(df_popu.shape)

211
(211, 6)


In [521]:
df_popu['BMI']=np.array(BMI_list)

In [522]:
df_popu

,patient_id,gender,race,age,身高,体重,BMI
0,6873870,女,汉族,28,153,52,22.21
3,547307,女,汉族,25,162,52,19.81
6,6949939,女,汉族,27,160,45,17.58
9,7765586,女,汉族,16,168,82,29.05
11,369329,女,汉族,33,159,67,26.50
...,...,...,...,...,...,...,...
938,8281006,男,汉族,14,168,52,18.42
947,8325164,男,汉族,30,167,79,28.33
951,7411504,女,汉族,17,163,57,21.45
953,8334957,男,汉族,14,166,49,17.78


## 合并人口学特征

In [523]:
df_VPA_popu=pd.merge(df_VPA_exclusion,df_popu,on=['patient_id'],how='left')

In [524]:
print(df_VPA_popu.shape)
print(df_VPA_popu['patient_id'].nunique())
print(df_VPA_popu['case_no'].nunique())

(1861, 24)
112
121


In [525]:
# 合并人口学特征
df_VPA_popu.to_excel(project_path+'/data/processed_data/df_3_合并人口学特征.xlsx')

# 提取安全性指标

## 消化道紊乱（恶心、胃疼、消化不良）

## 皮疹

## 高氨血症（血氨NH3>60）

## 血小板<100或降低超过50%

## 白细胞减少

## 肝功异常

## 头痛、嗜睡

## 骨髓抑制检测转置

In [ ]:
# 扩展新列
df_VPA_dosage.loc[:,'血红蛋白_before']=np.nan
df_VPA_dosage.loc[:,'血红蛋白_3d']=np.nan
df_VPA_dosage.loc[:,'血红蛋白_7d']=np.nan
df_VPA_dosage.loc[:,'血红蛋白_14d']=np.nan
df_VPA_dosage.loc[:,'白细胞_before']=np.nan
df_VPA_dosage.loc[:,'白细胞_3d']=np.nan
df_VPA_dosage.loc[:,'白细胞_7d']=np.nan
df_VPA_dosage.loc[:,'白细胞_14d']=np.nan
df_VPA_dosage.loc[:,'粒细胞_before']=np.nan
df_VPA_dosage.loc[:,'粒细胞_3d']=np.nan
df_VPA_dosage.loc[:,'粒细胞_7d']=np.nan
df_VPA_dosage.loc[:,'粒细胞_14d']=np.nan
df_VPA_dosage.loc[:,'血小板_before']=np.nan
df_VPA_dosage.loc[:,'血小板_3d']=np.nan
df_VPA_dosage.loc[:,'血小板_7d']=np.nan
df_VPA_dosage.loc[:,'血小板_14d']=np.nan

In [ ]:
df_VPA_dosage.shape

In [ ]:
# 将一次用药的不同检测装置到一行中，比如WBC_before,WBC_3d,WBC_7d
temp_list=[]
for i in df_VPA_dosage['case_no'].unique():
    temp=df_VPA_dosage[df_VPA_dosage['case_no']==i]
    temp=temp.sort_values(['case_no','start_datetime'])
    temp=temp.reset_index(drop=True)
    # 血红蛋白检测
    temp_HGB=df_test_HGB[df_test_HGB['case_no']==i]
    temp_HGB=temp_HGB.sort_values(['case_no','test_date'])
    temp_HGB=temp_HGB.reset_index(drop=True)
    # 白细胞检测
    temp_WBC=df_test_WBC[df_test_WBC['case_no']==i]
    temp_WBC=temp_WBC.sort_values(['case_no','test_date'])
    temp_WBC=temp_WBC.reset_index(drop=True)
    # 粒细胞检测
    temp_IG=df_test_IG[df_test_IG['case_no']==i]
    temp_IG=temp_IG.sort_values(['case_no','test_date'])
    temp_IG=temp_IG.reset_index(drop=True)
    # 血小板检测
    temp_PLT=df_test_PLT[df_test_PLT['case_no']==i]
    temp_PLT=temp_PLT.sort_values(['case_no','test_date'])
    temp_PLT=temp_PLT.reset_index(drop=True)
    # 骨髓抑制是降低指标，应该取用药后第一次检测。
    for j in range(temp.shape[0]):
        # 血红蛋白检测转置
        for k in range(temp_HGB.shape[0]):
            x=(temp_HGB.loc[k,'test_date']-temp.loc[j,'start_datetime']).days
            if (x>=-3)&(x<0):
                temp.loc[j,'血红蛋白_before']=temp_HGB.loc[k,'test_result']
            if (x>=1)&(x<=4):
                if pd.isnull(temp.loc[j,'血红蛋白_3d']):
                    temp.loc[j,'血红蛋白_3d']=temp_HGB.loc[k,'test_result']
            if (x>=6)&(x<=8):
                if pd.isnull(temp.loc[j,'血红蛋白_7d']):
                    temp.loc[j,'血红蛋白_7d']=temp_HGB.loc[k,'test_result']
            if (x>=13)&(x<=15):
                if pd.isnull(temp.loc[j,'血红蛋白_14d']):
                    temp.loc[j,'血红蛋白_14d']=temp_HGB.loc[k,'test_result']
        # 白细胞检测转置
        for k in range(temp_WBC.shape[0]):
            x=(temp_WBC.loc[k,'test_date']-temp.loc[j,'start_datetime']).days
            if (x>=-3)&(x<0):
                temp.loc[j,'白细胞_before']=temp_WBC.loc[k,'test_result']
            if (x>=1)&(x<=4):
                if pd.isnull(temp.loc[j,'白细胞_3d']):
                    temp.loc[j,'白细胞_3d']=temp_WBC.loc[k,'test_result']
            if (x>=6)&(x<=8):
                if pd.isnull(temp.loc[j,'白细胞_7d']):
                    temp.loc[j,'白细胞_7d']=temp_WBC.loc[k,'test_result']
            if (x>=13)&(x<=15):
                if pd.isnull(temp.loc[j,'白细胞_14d']):
                    temp.loc[j,'白细胞_14d']=temp_WBC.loc[k,'test_result']
        # 粒细胞检测转置
        for k in range(temp_IG.shape[0]):
            x=(temp_IG.loc[k,'test_date']-temp.loc[j,'start_datetime']).days
            if (x>=-3)&(x<0):
                temp.loc[j,'粒细胞_before']=temp_IG.loc[k,'test_result']
            if (x>=1)&(x<=4):
                if pd.isnull(temp.loc[j,'粒细胞_3d']):
                    temp.loc[j,'粒细胞_3d']=temp_IG.loc[k,'test_result']
            if (x>=6)&(x<=8):
                if pd.isnull(temp.loc[j,'粒细胞_7d']):
                    temp.loc[j,'粒细胞_7d']=temp_IG.loc[k,'test_result']
            if (x>=13)&(x<=15):
                if pd.isnull(temp.loc[j,'粒细胞_14d']):
                    temp.loc[j,'粒细胞_14d']=temp_IG.loc[k,'test_result']
        # 血小板检测转置
        for k in range(temp_PLT.shape[0]):
            x=(temp_PLT.loc[k,'test_date']-temp.loc[j,'start_datetime']).days
            if (x>=-3)&(x<0):
                temp.loc[j,'血小板_before']=temp_PLT.loc[k,'test_result']
            if (x>=1)&(x<=4):
                if pd.isnull(temp.loc[j,'血小板_3d']):
                    temp.loc[j,'血小板_3d']=temp_PLT.loc[k,'test_result']
            if (x>=6)&(x<=8):
                if pd.isnull(temp.loc[j,'血小板_7d']):
                    temp.loc[j,'血小板_7d']=temp_PLT.loc[k,'test_result']
            if (x>=13)&(x<=15):
                if pd.isnull(temp.loc[j,'血小板_14d']):
                    temp.loc[j,'血小板_14d']=temp_PLT.loc[k,'test_result']
    temp_list.append(temp)

In [ ]:
df_VPA_bmd_test_T=temp_list[0]
for j in range(1,len(temp_list)):
    df_VPA_bmd_test_T=pd.concat([df_VPA_bmd_test_T,temp_list[j]],axis=0)
df_VPA_bmd_test_T=df_VPA_bmd_test_T.reset_index(drop=True)
del temp_list

In [ ]:
df_VPA_bmd_test_T[df_VPA_bmd_test_T['case_no']=='786473']

In [ ]:
# 排序
df_VPA_bmd_test_T=df_VPA_bmd_test_T.sort_values(['case_no','start_datetime'])
df_VPA_bmd_test_T=df_VPA_bmd_test_T.reset_index(drop=True)

In [ ]:
print(df_VPA_bmd_test_T.shape)
print(df_VPA_bmd_test_T['case_no'].nunique())

In [ ]:
# 保存甲氨蝶呤骨髓抑制检测
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_5.1_甲氨蝶呤骨髓抑制检测转置.xlsx')
df_VPA_bmd_test_T.to_excel(writer)
writer.save()

## 骨髓抑制检测值0-1转换

In [ ]:
# 根据本善提供的指标对骨髓抑制指标进行二进制转换
df_VPA_bmd_test_binary=df_VPA_bmd_test_T.copy()
# 判断血红蛋白指标
df_VPA_bmd_test_binary['血红蛋白_before_B']=df_VPA_bmd_test_binary['血红蛋白_before'].apply(lambda x: x if pd.isnull(x) else
                                                                                  1 if float(x)<=109 else 0)
df_VPA_bmd_test_binary['血红蛋白_3d_B']=df_VPA_bmd_test_binary['血红蛋白_3d'].apply(lambda x: x if pd.isnull(x) else
                                                                                  1 if float(x)<=109 else 0)
df_VPA_bmd_test_binary['血红蛋白_7d_B']=df_VPA_bmd_test_binary['血红蛋白_7d'].apply(lambda x: x if pd.isnull(x) else
                                                                              1 if float(x)<=109 else 0)
df_VPA_bmd_test_binary['血红蛋白_14d_B']=df_VPA_bmd_test_binary['血红蛋白_14d'].apply(lambda x: x if pd.isnull(x) else
                                                                              1 if float(x)<=109 else 0)
# 判断白细胞指标
df_VPA_bmd_test_binary['白细胞_before_B']=df_VPA_bmd_test_binary['白细胞_before'].apply(lambda x: x if pd.isnull(x) else
                                                                              1 if float(x)<=3.9 else 0)
df_VPA_bmd_test_binary['白细胞_3d_B']=df_VPA_bmd_test_binary['白细胞_3d'].apply(lambda x: x if pd.isnull(x) else
                                                                              1 if float(x)<=3.9 else 0)
df_VPA_bmd_test_binary['白细胞_7d_B']=df_VPA_bmd_test_binary['白细胞_7d'].apply(lambda x: x if pd.isnull(x) else
                                                                              1 if float(x)<=3.9 else 0)
df_VPA_bmd_test_binary['白细胞_14d_B']=df_VPA_bmd_test_binary['白细胞_14d'].apply(lambda x: x if pd.isnull(x) else
                                                                              1 if float(x)<=3.9 else 0)
# 判断粒细胞指标
df_VPA_bmd_test_binary['粒细胞_before_B']=df_VPA_bmd_test_binary['粒细胞_before'].apply(lambda x: x if pd.isnull(x) else
                                                                              1 if float(x)<=1.9 else 0)
df_VPA_bmd_test_binary['粒细胞_3d_B']=df_VPA_bmd_test_binary['粒细胞_3d'].apply(lambda x: x if pd.isnull(x) else
                                                                              1 if float(x)<=1.9 else 0)
df_VPA_bmd_test_binary['粒细胞_7d_B']=df_VPA_bmd_test_binary['粒细胞_7d'].apply(lambda x: x if pd.isnull(x) else
                                                                              1 if float(x)<=1.9 else 0)
df_VPA_bmd_test_binary['粒细胞_14d_B']=df_VPA_bmd_test_binary['粒细胞_14d'].apply(lambda x: x if pd.isnull(x) else
                                                                              1 if float(x)<=1.9 else 0)
# 判断血小板指标
df_VPA_bmd_test_binary['血小板_before_B']=df_VPA_bmd_test_binary['血小板_before'].apply(lambda x: x if pd.isnull(x) else
                                                                              1 if float(x)<=99 else 0)
df_VPA_bmd_test_binary['血小板_3d_B']=df_VPA_bmd_test_binary['血小板_3d'].apply(lambda x: x if pd.isnull(x) else
                                                                              1 if float(x)<=99 else 0)
df_VPA_bmd_test_binary['血小板_7d_B']=df_VPA_bmd_test_binary['血小板_7d'].apply(lambda x: x if pd.isnull(x) else
                                                                              1 if float(x)<=99 else 0)
df_VPA_bmd_test_binary['血小板_14d_B']=df_VPA_bmd_test_binary['血小板_14d'].apply(lambda x: x if pd.isnull(x) else
                                                                              1 if float(x)<=99 else 0)

In [ ]:
df_VPA_bmd_test_binary

In [ ]:
print(df_VPA_bmd_test_binary.shape)
print(df_VPA_bmd_test_binary['patient_id'].nunique())

In [ ]:
# 保存甲氨蝶呤骨髓抑制检测
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_5.2_甲氨蝶呤骨髓抑制检测0-1转置.xlsx')
df_VPA_bmd_test_binary.to_excel(writer)
writer.save()

## 判断骨髓抑制

In [ ]:
# 如果同时出现2个指标降低，则认为发生
for i in range(df_VPA_bmd_test_binary.shape[0]):
    # 检测甲氨蝶呤服药前骨髓抑制
    list_before=list(df_VPA_bmd_test_binary.loc[i,['血红蛋白_before_B','白细胞_before_B','粒细胞_before_B','血小板_before_B']])
    if list_before.count(1) >=2:
        df_VPA_bmd_test_binary.loc[i,'bmd_before']=1
    else:
        df_VPA_bmd_test_binary.loc[i,'bmd_before']=0
    # 检测甲氨蝶呤服药第3天骨髓抑制
    list_before=list(df_VPA_bmd_test_binary.loc[i,['血红蛋白_3d_B','白细胞_3d_B','粒细胞_3d_B','血小板_3d_B']])
    if list_before.count(1) >=2:
        df_VPA_bmd_test_binary.loc[i,'bmd_3d']=1
    else:
        df_VPA_bmd_test_binary.loc[i,'bmd_3d']=0
    # 检测甲氨蝶呤服药第7天骨髓抑制
    list_before=list(df_VPA_bmd_test_binary.loc[i,['血红蛋白_7d_B','白细胞_7d_B','粒细胞_7d_B','血小板_7d_B']])
    if list_before.count(1) >=2:
        df_VPA_bmd_test_binary.loc[i,'bmd_7d']=1
    else:
        df_VPA_bmd_test_binary.loc[i,'bmd_7d']=0
    # 检测甲氨蝶呤服药第14天骨髓抑制
    list_before=list(df_VPA_bmd_test_binary.loc[i,['血红蛋白_14d_B','白细胞_14d_B','粒细胞_14d_B','血小板_14d_B']])
    if list_before.count(1) >=2:
        df_VPA_bmd_test_binary.loc[i,'bmd_14d']=1
    else:
        df_VPA_bmd_test_binary.loc[i,'bmd_14d']=0

In [ ]:
df_VPA_bmd_test_binary

In [ ]:
# 排序
df_VPA_bmd_test_binary=df_VPA_bmd_test_binary.sort_values(['patient_id','start_datetime'])
df_VPA_bmd_test_binary=df_VPA_bmd_test_binary.reset_index(drop=True)

In [ ]:
# 保存甲氨蝶呤骨髓抑制统计
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_5.3_判断是否出现骨髓抑制.xlsx')
df_VPA_bmd_test_binary.to_excel(writer)
writer.save()

## 提取第一次入院第一次出现骨髓抑制

### 提取第一次入院时的骨髓抑制

In [ ]:
# 取第一次入院时的用药信息
df_VPA_bmd_test_binary=df_VPA_bmd_test_binary.sort_values(['patient_id','start_datetime'])
df_VPA_bmd_test_first=df_VPA_bmd_test_binary.drop_duplicates(subset=['patient_id'],keep='first')
df_VPA_bmd_test_first=df_VPA_bmd_test_first.reset_index(drop=True)

In [ ]:
print(df_VPA_bmd_test_first.shape)
print(df_VPA_bmd_test_first['patient_id'].nunique())

In [ ]:
# 保存第一次入院时甲氨蝶呤骨髓抑制统计
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_5.4.1_提取第一次入院出现骨髓抑制.xlsx')
df_VPA_bmd_test_first.to_excel(writer)
writer.save()

In [ ]:
# 统计第一次入院前就骨髓抑制
df_VPA_bmd_before=df_VPA_bmd_test_first[df_VPA_bmd_test_first['bmd_before']==1]

In [ ]:
print(df_VPA_bmd_before.shape)
print(df_VPA_bmd_before['patient_id'].nunique())

In [ ]:
# 统计第一次用药3天后骨髓抑制
df_VPA_bmd_3d=df_VPA_bmd_test_first[(df_VPA_bmd_test_first['bmd_3d']==1)&
                                                             (df_VPA_bmd_test_first['bmd_before']==0)]

In [ ]:
print(df_VPA_bmd_3d.shape)
print(df_VPA_bmd_3d['patient_id'].nunique())

In [ ]:
# 统计第一次用药7天后骨髓抑制
df_VPA_bmd_7d=df_VPA_bmd_test_first[(df_VPA_bmd_test_first['bmd_7d']==1)&
                                                              (df_VPA_bmd_test_first['bmd_3d']==0)&
                                                             (df_VPA_bmd_test_first['bmd_before']==0)]

In [ ]:
print(df_VPA_bmd_7d.shape)
print(df_VPA_bmd_7d['patient_id'].nunique())

In [ ]:
# 统计第一次用药14天后骨髓抑制
df_VPA_bmd_14d=df_VPA_bmd_test_first[(df_VPA_bmd_test_first['bmd_14d']==1)&
                                                               (df_VPA_bmd_test_first['bmd_7d']==0)&
                                                              (df_VPA_bmd_test_first['bmd_3d']==0)&
                                                             (df_VPA_bmd_test_first['bmd_before']==0)]

In [ ]:
print(df_VPA_bmd_14d.shape)
print(df_VPA_bmd_14d['patient_id'].nunique())

In [ ]:
# 统计第一次用药后没有出现骨髓抑制
df_VPA_bmd_no=df_VPA_bmd_test_first[(df_VPA_bmd_test_first['bmd_14d']==0)&
                                                               (df_VPA_bmd_test_first['bmd_7d']==0)&
                                                              (df_VPA_bmd_test_first['bmd_3d']==0)&
                                                             (df_VPA_bmd_test_first['bmd_before']==0)]

In [ ]:
print(df_VPA_bmd_no.shape)
print(df_VPA_bmd_no['patient_id'].nunique())

### 提取第一次发生骨髓抑制

In [ ]:
# 取7天内天作为不良反应发生时间提取范围
df_VPA_bmd=pd.concat([df_VPA_bmd_3d,df_VPA_bmd_7d],axis=0)
df_VPA_bmd=df_VPA_bmd.reset_index(drop=True)

In [ ]:
print(df_VPA_bmd.shape)
print(df_VPA_bmd['patient_id'].nunique())

In [ ]:
df_VPA_bmd['age'].describe()

In [ ]:
# 保存第一次入院发生不良反应
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_5.4.2_提取VPA用药后第一次出现骨髓抑制.xlsx')
df_VPA_bmd.to_excel(writer)
writer.save()

### 提取第一次发生骨髓抑制时间

In [ ]:
# 计算第一次用药到第一次发生不良反应时间间隔=test_date - start_datetime
temp_list=[]
for i in df_VPA_bmd['case_no'].unique():
    # print(i)
    temp=df_VPA_bmd[df_VPA_bmd['case_no']==i]
    temp=temp.reset_index(drop=True)
    temp_bmd=df_test_bmd_low[(df_test_bmd_low['case_no']==i) &
        (df_test_bmd_low['test_date']>=temp.loc[0,'start_datetime']+datetime.timedelta(days=1))]
    temp_bmd=temp_bmd.sort_values(['test_date'])
    temp_bmd=temp_bmd.reset_index(drop=True)
    for j in range(1,temp_bmd.shape[0]):
        # 如果出现2个降低的不同指标，则认为发生骨髓抑制，计算发生时间
        if temp_bmd.loc[0,'project_name'] != temp_bmd.loc[j,'project_name']:
            t = (temp_bmd.loc[j,'test_date']-temp.loc[0,'start_datetime']).days
            # 发生不良反应的时间
            temp.loc[0,'bmd_time']=temp_bmd.loc[j,'test_date']
            # 服药到发生不良反应的时间间隔
            temp.loc[0,'start_bmd_time']=t
            break
    temp_list.append(temp)

In [ ]:
df_VPA_bmd_time=temp_list[0]
for j in range(1,len(temp_list)):
    df_VPA_bmd_time=pd.concat([df_VPA_bmd_time,temp_list[j]],axis=0)
df_VPA_bmd_time=df_VPA_bmd_time.reset_index(drop=True)
del temp_list

In [ ]:
df_VPA_bmd_time=df_VPA_bmd_time.sort_values(['case_no','start_datetime'])
df_VPA_bmd_time=df_VPA_bmd_time.reset_index(drop=True)

In [ ]:
# 保存第一次入院第一次发生不良反应时间间隔
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_5.4.3_计算VPA用药-发生骨髓抑制时间间隔.xlsx')
df_VPA_bmd_time.to_excel(writer)
writer.save()

## 骨髓抑制打标签

In [ ]:
# 用药7天内新出现骨髓抑制算发生不良反应-1，其他的情况为0
df_VPA_bmd_time['bmd_label']=1

In [ ]:
df_VPA_bmd_0=pd.concat([df_VPA_bmd_before,df_VPA_bmd_14d],axis=0)
df_VPA_bmd_0=pd.concat([df_VPA_bmd_0,df_VPA_bmd_no],axis=0)
df_VPA_bmd_0['bmd_label']=0

In [ ]:
# 将无不良反应的bmd_time定义为服药后第七天
df_VPA_bmd_0['bmd_time']=df_VPA_bmd_0['start_datetime'].apply(lambda x: x+datetime.timedelta(days=7))

In [ ]:
df_VPA_bmd_label=pd.concat([df_VPA_bmd_time,df_VPA_bmd_0],axis=0)
df_VPA_bmd_label=df_VPA_bmd_label.reset_index(drop=True)

In [ ]:
print(df_VPA_bmd_label.shape)
print(df_VPA_bmd_label['patient_id'].nunique())

In [ ]:
# 保存甲氨蝶呤骨髓抑制分组
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_5.5_甲氨蝶呤骨髓抑制打标签.xlsx')
df_VPA_bmd_label.to_excel(writer)
writer.save()

# 添加丙戊酸TDM检测

## 提取tdm检测数据

In [300]:
# 提取用药后不良反应发生前的甲氨蝶呤血药浓度检测
df_test_VPA=df_test[df_test['project_name'].str.contains('丙戊酸|VPA')]
df_test_VPA=df_test_VPA.reset_index(drop=True)

In [301]:
df_test_VPA['tdm_date']=df_test_VPA['test_date'].apply(lambda x: str(x).split(' ')[0])

In [302]:
print(df_test_VPA.shape)
print(df_test_VPA['patient_id'].nunique())

(754, 14)
358


In [303]:
# 提取患者的tdm检测
df_test_VPA=df_test_VPA[df_test_VPA['patient_id'].isin(df_VPA['patient_id'].unique())]
df_test_VPA=df_test_VPA.sort_values(['patient_id','test_date'])
df_test_VPA=df_test_VPA.reset_index(drop=True)

In [304]:
print(df_test_VPA.shape)
print(df_test_VPA['patient_id'].nunique())

(515, 14)
212


In [305]:
# 保存丙戊酸血药浓度检测
df_test_VPA.to_excel(project_path+'/data/processed_data/df_5.1_test_VPA.xlsx')

## 合并tdm检测数据

In [701]:
import datetime
# 合并甲氨蝶呤tdm检测信息
# 如果tdm用药前3天含有非稳态用药，删除；否则，保留。
temp_list=[]
for i in np.unique(df_VPA_popu['case_no']):
    # 稳态前 丙戊酸 temp
    temp_before=df_VPA_modification[df_VPA_modification['case_no']==i]
    temp_before=temp_before.sort_values(['patient_id','case_no','分裂日期'])
    temp_before=temp_before.reset_index(drop=True)
    # 稳态 丙戊酸 temp
    temp_steady=df_VPA_popu[df_VPA_popu['case_no']==i]
    temp_steady=temp_steady.sort_values(['patient_id','case_no','分裂日期'])
    temp_steady=temp_steady.reset_index(drop=True)
    # tdm temp
    temp_tdm=df_test_VPA[df_test_VPA['case_no']==i]
    temp_tdm=temp_tdm.sort_values(['patient_id','case_no','test_date'])
    temp_tdm=temp_tdm.reset_index(drop=True)
    # 判断tdm前3天内是否有非稳态用药
    for j in range(temp_tdm.shape[0]):
        # 提取tdm检测日期
        tdm_date=pd.to_datetime(temp_tdm.loc[j,'tdm_date'])
        # 提取稳态前tdm前3天用药
        temp_before_drug=temp_before[(pd.to_datetime(temp_before['分裂日期'])<=tdm_date-datetime.timedelta(days=1))&
                                    (pd.to_datetime(temp_before['分裂日期'])>=tdm_date-datetime.timedelta(days=3))]
        # 提取稳态时tdm前3天用药
        temp_steady_drug=temp_steady[(pd.to_datetime(temp_steady['分裂日期'])<=tdm_date-datetime.timedelta(days=1))&
                            (pd.to_datetime(temp_steady['分裂日期'])>=tdm_date-datetime.timedelta(days=3))]
        # 如果稳态前和稳态时用药有变化，则舍弃tdm数据，否则，并入到VPA用药数据中
        if temp_before_drug.shape[0] != temp_steady_drug.shape[0]:
            continue
        else:
            if temp_steady_drug.shape[0]>0:
                # 获取最近用药的索引
                index=temp_steady_drug.index.tolist()[-1]
                temp_steady_drug.loc[index,'tdm_date']=temp_tdm.loc[j,'tdm_date']
                temp_steady_drug.loc[index,'test_result']=temp_tdm.loc[j,'test_result']
        temp_list.append(temp_steady_drug)

In [702]:
temp_tdm

,test_record_id,patient_id,case_no,test_date,clinical_diagnosis,sample_type,sample_time,project_name,test_result,result_unit,is_normal,refer_scope,synonym,tdm_date
0,800008575295||Y006||1,8374053,9963165,2020-08-14 11:18:00,双相情感障碍，目前为混合性发作,血浆,NaT,丙戊酸药物浓度监测,79.70,ug/ml,N,50.00-100.00,VPA,2020-08-14


In [703]:
df_VPA_tdm = pd.concat(temp_list,axis=0).reset_index(drop=True)
del temp_list

In [704]:
print(df_VPA_tdm.shape)
print(df_VPA_tdm['patient_id'].nunique())
print(df_VPA_tdm['case_no'].nunique())

(381, 26)
89
96


In [705]:
df_VPA_tdm.columns

Index(['patient_id', 'case_no', 'drug_name', 'amount', 'drug_spec', 'dosage',
       'frequency', 'start_datetime', 'end_datetime', 'statusdesc', '日剂量',
       'start_date', 'end_date', 'long_d_order', '分裂日期', '起始标签', '时间标签',
       '连续标签', 'gender', 'race', 'age', '身高', '体重', 'BMI', 'tdm_date',
       'test_result'],
      dtype='object')

In [706]:
# 保存添加tdm检测的甲氨蝶呤用药
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_5.2_添加tdm检测.xlsx')
df_VPA_tdm.to_excel(writer)
writer.save()

## 删除tdm为空的数据

In [743]:
df_VPA_tdm=df_VPA_tdm[df_VPA_tdm['test_result'].notnull()].reset_index(drop=True)

In [744]:
print(df_VPA_tdm.shape)
print(df_VPA_tdm['patient_id'].nunique())
print(df_VPA_tdm['case_no'].nunique())

(127, 26)
89
96


In [745]:
# 保存tdm非空的用药
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_5.3_tdm非空用药.xlsx')
df_VPA_tdm.to_excel(writer)
writer.save()

PermissionError: [Errno 13] Permission denied: 'D:\\PycharmProjects\\VPA_southern_hospital/data/processed_data/df_5.3_tdm非空用药.xlsx'

# 增加联合用药

## 提取联合用药

In [710]:
# 提取患者的联合用药
df_drug_other=df_doctor_order[~ df_doctor_order['drug_name'].str.contains('丙戊酸|葡萄糖|氯化钠')]
# 患者限定
df_drug_other=df_drug_other[df_drug_other['case_no'].isin(df_VPA_tdm['case_no'].unique())]
df_drug_other=df_drug_other.reset_index(drop=True)

In [711]:
print(df_drug_other.shape)
print(df_drug_other['patient_id'].nunique())

(13620, 10)
89


In [712]:
df_drug_other

,patient_id,case_no,drug_name,amount,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc
0,6873870,10001302,齐拉西酮冻干粉针(江都医院)[30mg*1瓶],1.0,30mg*1瓶,20mg,ONCE,2020-08-15 18:59:57,NaT,核实
1,6873870,10001302,行为矫正治疗,20.0,None,None,1/日,2020-08-15 19:45:12,2020-09-04 11:38:00,停止
2,6873870,10001302,静脉输液第二组起每组,3.0,None,None,1/日,2020-08-17 11:24:51,2020-08-20 10:11:04,停止
3,6873870,10001302,脑苷肌肽注射液(欧迪美)[2ml*1支],6.0,2ml*1支,4ml,1/日,2020-08-17 11:25:27,2020-08-20 10:11:09,停止
4,6873870,10001302,静脉输液第二组起每组,3.0,None,None,1/日,2020-08-17 11:25:27,2020-08-20 10:11:09,停止
...,...,...,...,...,...,...,...,...,...,...
13615,6865481,9910592,别嘌醇缓释胶囊(奥迈必利)[0.25g*10粒],1.0,0.25g*10粒,0.25g,1/日,2020-08-07 11:07:19,2020-08-11 11:14:34,停止
13616,6865481,9910592,别嘌醇缓释胶囊(奥迈必利)[0.25g*10粒],1.0,0.25g*10粒,0.25g,ONCE,2020-08-07 11:07:19,NaT,核实
13617,6865481,9910592,单人房B级(江高精神心理科),1.0,None,None,None,NaT,NaT,核实
13618,6865481,9910592,住院诊查费,1.0,None,None,None,NaT,NaT,核实


In [808]:
df_drug_other.to_excel(project_path+'/data/processed_data/df_6.1_drug_other.xlsx')

## 时间限定联合用药

In [810]:
temp

,patient_id,case_no,drug_name,amount,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc
0,6873870,10001302,行为矫正治疗,20.0,None,None,1/日,2020-08-15 19:45:12,2020-09-04 11:38:00,停止
1,6873870,10001302,常规测血压、脉搏、呼吸,20.0,None,None,1/日,2020-08-15 19:45:12,2020-09-04 11:38:00,停止
2,6873870,10001302,普食,20.0,None,None,None,2020-08-15 19:45:12,2020-09-04 11:38:00,停止
3,6873870,10001302,二级护理,20.0,None,None,None,2020-08-15 19:45:12,2020-09-04 11:38:00,停止
4,6873870,10001302,按心理科常规护理,20.0,None,None,1/日,2020-08-15 19:45:12,2020-09-04 11:38:00,停止
5,6873870,10001302,陪人一名,20.0,None,None,1/日,2020-08-15 19:45:12,2020-09-04 11:38:00,停止
6,6873870,10001302,精神科监护,20.0,None,None,1/日,2020-08-15 19:45:12,2020-09-04 11:38:00,停止
7,6873870,10001302,脑电生物反馈治疗,19.0,None,None,1/晚,2020-08-15 19:45:12,2020-09-04 11:38:00,停止
8,6873870,10001302,抗精神病药物治疗监测,20.0,None,None,1/日,2020-08-15 19:45:12,2020-09-04 11:38:00,停止
9,6873870,10001302,行为观察和治疗,20.0,None,None,1/日,2020-08-15 19:45:12,2020-09-04 11:38:00,停止


In [811]:
# 联合用药时间限定：用药前3天内-tdm检测前，各自反集取交集
temp_list=[]
for i in df_drug_other['case_no'].unique():
#     print(i)
#     if i != '10001302':
#         break
    temp=df_drug_other[df_drug_other['case_no']==i]
    temp=temp.reset_index(drop=True)
    temp['start_date']=temp['start_datetime'].apply(lambda x: pd.to_datetime(str(x).split(' ')[0])) # 分裂联合日期
    temp['end_date']=temp['end_datetime'].apply(lambda x: pd.to_datetime(str(x).split(' ')[0])) # 分裂联合用药日期
#     break
    # VPA用药时间
    temp_drug=df_VPA_tdm[df_VPA_tdm['case_no']==i]
    temp_drug=temp_drug.reset_index(drop=True)
    for j in range(temp_drug.shape[0]):
        drug_time=temp_drug.loc[j,'start_date']
        tdm_time=temp_drug.loc[j,'tdm_date']
        # 提取用药前3天内-tdm前的联合用药。因为包含临时用药，所以
        temp=temp[(temp['end_date']>=pd.to_datetime(drug_time)-datetime.timedelta(days=3)) &
                 (temp['start_date']<=pd.to_datetime(tdm_time))]
    #     print(temp)
        temp=temp.sort_values(['start_date'])
        temp_list.append(temp)

In [812]:
df_drug_other = pd.concat(temp_list,axis=0).reset_index(drop=True)
del temp_list

In [813]:
df_drug_other

,patient_id,case_no,drug_name,amount,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc,start_date,end_date
0,6873870,10001302,行为矫正治疗,20.0,None,None,1/日,2020-08-15 19:45:12,2020-09-04 11:38:00,停止,2020-08-15,2020-09-04
1,6873870,10001302,冲动行为干预治疗,6.0,None,None,1/日,2020-08-15 20:52:20,2020-08-21 20:10:33,停止,2020-08-15,2020-08-21
2,6873870,10001302,暗示治疗,20.0,None,None,1/日,2020-08-15 19:45:12,2020-09-04 11:38:00,停止,2020-08-15,2020-09-04
3,6873870,10001302,行为观察和治疗,20.0,None,None,1/日,2020-08-15 19:45:12,2020-09-04 11:38:00,停止,2020-08-15,2020-09-04
4,6873870,10001302,抗精神病药物治疗监测,20.0,None,None,1/日,2020-08-15 19:45:12,2020-09-04 11:38:00,停止,2020-08-15,2020-09-04
...,...,...,...,...,...,...,...,...,...,...,...,...
3359,6865481,9910592,静脉输液(住院),20.0,None,None,1/日,2020-08-06 15:09:07,2020-08-26 11:36:33,停止,2020-08-06,2020-08-26
3360,6865481,9910592,奥氮平口崩片[10mg*7片],2.0,10mg*7片,15mg,1/晚,2020-08-11 11:17:34,2020-08-17 10:13:39,停止,2020-08-11,2020-08-17
3361,6865481,9910592,冲动行为干预治疗,2.0,None,None,1/日,2020-08-14 22:07:22,2020-08-16 10:56:13,停止,2020-08-14,2020-08-16
3362,6865481,9910592,上约束带,2.0,None,None,1/日,2020-08-14 22:07:22,2020-08-16 10:56:13,停止,2020-08-14,2020-08-16


In [814]:
# 时间限定的联合用药
df_drug_other.to_excel(project_path+'/data/processed_data/df_6.2_非空drug_other.xlsx')

## 统计联合用药频数

In [815]:
df_drug_other_name=pd.DataFrame(df_drug_other.drug_name.value_counts())
# 保存联合用药名
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_6.3_联合用药名.xlsx')
df_drug_other_name.to_excel(writer)
writer.save()

## 合并联合用药

In [ ]:
# 主药：多柔比星、多柔比星脂质体、顺铂、异环磷酰胺、依托泊苷
df_ADM = df_drug_other[df_drug_other['drug_name'].str.contains('注射用盐酸多柔比星')]
df_IFO = df_drug_other[df_drug_other['drug_name'].str.contains('异环磷酰胺|IFO')]
df_CDP = df_drug_other[df_drug_other['drug_name'].str.contains('顺铂|CDP')]
df_VP = df_drug_other[df_drug_other['drug_name'].str.contains('依托泊苷|VP')]
df_drug_zhi = df_drug_other[df_drug_other['drug_name'].str.contains('多柔比星脂质体')]

# 辅助用药：头孢类、促红素、抗凝血、吡柔比星、保肝药、胃癌药
df_drug_toubao =  df_drug_other[df_drug_other['drug_name'].str.contains('头孢')]
df_drug_EPO = df_drug_other[df_drug_other['drug_name'].str.contains('促红素')]
df_drug_THP = df_drug_other[df_drug_other['drug_name'].str.contains('吡柔比星|吡柔吡星|THP')]
df_drug_kang = df_drug_other[df_drug_other['drug_name'].str.contains('美司钠|肝素钠')]
df_drug_GSH = df_drug_other[df_drug_other['drug_name'].str.contains('谷胱甘肽')]
df_drug_ganmei = df_drug_other[df_drug_other['drug_name'].str.contains('异甘草酸镁')]
df_drug_PO4Na = df_drug_other[df_drug_other['drug_name'].str.contains('地塞米松磷酸钠')]
df_drug_Tro = df_drug_other[df_drug_other['drug_name'].str.contains('托烷司琼')]
df_drug_gangan = df_drug_other[df_drug_other['drug_name'].str.contains('甘草酸苷')]
df_drug_VCR = df_drug_other[df_drug_other['drug_name'].str.contains('长春新碱')]
df_drug_ppt = df_drug_other[df_drug_other['drug_name'].str.contains('多烯磷脂酰')]

In [ ]:
# 合并联合用药
df_VPA_drug_other=df_VPA_history.copy()
df_VPA_drug_other['多柔比星']=df_VPA_drug_other['case_no'].astype('str').apply(lambda x: 1 if x in df_ADM['case_no'].unique() else 0)
df_VPA_drug_other['异环磷酰胺']=df_VPA_drug_other['case_no'].astype('str').apply(lambda x: 1 if x in df_IFO['case_no'].unique() else 0)
df_VPA_drug_other['顺铂']=df_VPA_drug_other['case_no'].astype('str').apply(lambda x: 1 if x in df_CDP['case_no'].unique() else 0)
df_VPA_drug_other['依托泊苷']=df_VPA_drug_other['case_no'].astype('str').apply(lambda x: 1 if x in df_VP['case_no'].unique() else 0)
df_VPA_drug_other['多柔比星脂质体']=df_VPA_drug_other['case_no'].astype('str').apply(lambda x: 1 if x in df_drug_zhi['case_no'].unique() else 0)

df_VPA_drug_other['头孢类']=df_VPA_drug_other['case_no'].astype('str').apply(lambda x: 1 if x in df_drug_toubao['case_no'].unique() else 0)
df_VPA_drug_other['促红素']=df_VPA_drug_other['case_no'].astype('str').apply(lambda x: 1 if x in df_drug_EPO['case_no'].unique() else 0)
df_VPA_drug_other['吡柔吡星']=df_VPA_drug_other['case_no'].astype('str').apply(lambda x: 1 if x in df_drug_THP['case_no'].unique() else 0)
df_VPA_drug_other['抗凝血']=df_VPA_drug_other['case_no'].astype('str').apply(lambda x: 1 if x in df_drug_kang['case_no'].unique() else 0)

df_VPA_drug_other['谷胱甘肽']=df_VPA_drug_other['case_no'].astype('str').apply(lambda x: 1 if x in df_drug_GSH['case_no'].unique() else 0)
df_VPA_drug_other['异甘草酸镁']=df_VPA_drug_other['case_no'].astype('str').apply(lambda x: 1 if x in df_drug_ganmei['case_no'].unique() else 0)
df_VPA_drug_other['地塞米松磷酸钠']=df_VPA_drug_other['case_no'].astype('str').apply(lambda x: 1 if x in df_drug_PO4Na['case_no'].unique() else 0)
df_VPA_drug_other['托烷司琼']=df_VPA_drug_other['case_no'].astype('str').apply(lambda x: 1 if x in df_drug_Tro['case_no'].unique() else 0)
df_VPA_drug_other['甘草酸苷']=df_VPA_drug_other['case_no'].astype('str').apply(lambda x: 1 if x in df_drug_gangan['case_no'].unique() else 0)
df_VPA_drug_other['长春新碱']=df_VPA_drug_other['case_no'].astype('str').apply(lambda x: 1 if x in df_drug_VCR['case_no'].unique() else 0)
df_VPA_drug_other['多烯磷脂酰']=df_VPA_drug_other['case_no'].astype('str').apply(lambda x: 1 if x in df_drug_ppt['case_no'].unique() else 0)

In [ ]:
# 保存联合用药
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_6.4_合并联合用药.xlsx')
df_VPA_drug_other.to_excel(writer)
writer.save()

# 合并其他相关检测

## 提取其他检测

In [761]:
# 提取甲氨蝶呤患者的所有检测
df_test_other=df_test[df_test['case_no'].isin(df_VPA_tdm['case_no'].unique())]

In [762]:
# 提取患者的其他检测
df_test_other = df_test_other[~ df_test_other['project_name'].str.contains('丙戊酸')]
df_test_other = df_test_other.reset_index(drop=True)

In [763]:
print(df_test_other.shape)
print(df_test_other['patient_id'].nunique())

(14416, 13)
89


In [764]:
# 保存其他检测
df_test_other.to_csv(project_path+'/data/processed_data/df_7.1_test_other.csv')

## 统计其他检测名称频数

In [765]:
df_test_other_name=pd.DataFrame(df_test_other['project_name'].value_counts())

In [766]:
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_7.2_其他检测项.xlsx')
df_test_other_name.to_excel(writer)
writer.save()

## 合并其他检测

In [767]:
def judge_float(x):
    try:
        a=float(x)
        return a
    except:
        return np.NaN

In [768]:
df_VPA_tdm

,patient_id,case_no,drug_name,amount,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc,...,时间标签,连续标签,gender,race,age,身高,体重,BMI,tdm_date,test_result
0,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2020-08-24 10:37:31,2020-09-04 11:38:00,停止,...,0,17,女,汉族,28,153,52,22.21,2020-09-02,102.50
1,547307,10005574,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2020-08-16 16:41:46,2020-08-31 11:15:00,停止,...,1,9,女,汉族,25,162,52,19.81,2020-08-26,97.30
2,6949939,10019847,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2020-08-17 21:10:00,2020-09-03 08:26:00,停止,...,1,3,女,汉族,27,160,45,17.58,2020-08-21,78.20
3,369329,10028911,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2020-08-18 16:35:34,2020-09-02 08:34:00,停止,...,1,8,女,汉族,33,159,67,26.50,2020-08-27,86.40
4,8413999,10171606,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2020-09-04 10:50:11,2020-09-15 09:09:00,停止,...,0,11,女,汉族,23,160,63,24.61,2020-09-14,130.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,8029177,9739796,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2020-07-20 10:49:58,2020-07-27 11:02:39,停止,...,0,4,女,汉族,18,160,44,17.19,2020-07-24,148.20
123,8029177,9739796,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-07-27 11:07:39,2020-08-08 08:13:00,停止,...,0,14,女,汉族,18,160,44,17.19,2020-08-03,79.40
124,8325164,9812242,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.75,1,2020-08-03 10:43:56,2020-08-14 08:16:00,停止,...,0,9,男,汉族,30,167,79,28.33,2020-08-06,82.00
125,8325164,9812242,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.75,1,2020-08-03 10:43:56,2020-08-14 08:16:00,停止,...,0,16,男,汉族,30,167,79,28.33,2020-08-13,87.90


In [795]:
df_VPA_test_other=df_VPA_tdm.copy()
# 将用药后不良反应发生前的最近一次其他检测转置到一行中
for i in range(df_VPA_test_other.shape[0]):
    case_no=df_VPA_test_other.loc[i,'case_no']
    print(case_no)
    temp=df_VPA_test_other[df_VPA_test_other['case_no']==case_no]
    temp=temp.reset_index(drop=True)
    
    # case_no的其他检测
    temp_other= df_test_other[df_test_other['case_no']==case_no]
    temp_other=temp_other.reset_index(drop=True)
    temp_other['test_date']=temp_other['test_date'].apply(lambda x: pd.to_datetime(str(x).split(' ')[0])) # 分裂检测日期
    # 取用药后不良反应发生前的其他检测
    temp_other_1=temp_other[(temp_other['test_date']>=pd.to_datetime(temp.loc[0,'分裂日期'])+datetime.timedelta(days=1))&
                            (temp_other['test_date']<=temp.loc[0,'tdm_date'])]
    # 取最近的一次其他检测
    temp_other_1=temp_other_1.sort_values(['test_date'])
    temp_other_1=temp_other_1.drop_duplicates(['project_name'],keep='last')
    temp_other_1=temp_other_1.reset_index(drop=True)
    
    # 检验项转置为表头
    if temp_other_1.shape[0]>0:
        temp_other_1['test_result']=temp_other_1['test_result'].apply(judge_float)
        temp_other_pivot=temp_other_1.pivot_table('test_result',['case_no'], 'project_name')
        temp_other_pivot=temp_other_pivot.reset_index(drop=True)
        for j in temp_other_pivot.columns[1:]:
            df_VPA_test_other.loc[i,j]=temp_other_pivot.loc[0,j]
#     break
#     if case_no == '410308':
#         print(temp.loc[0,'start_datetime'])
#         print(temp.loc[0,'bmd_time'])
#         print(temp_other_1)
#         print(temp_other_2[temp_other_2['project_name']=='肌酐(干式)'])
#         break

10001302
10005574
10019847
10028911
10171606
11351405
11351405
13063464
4045163
4069083
4103620
4112018
4133350
4191397
4216082
4255479
4255479
4267891
4267891
4364618
4364618
4446056
4446680
4536465
4536465
4536465
4553217
4560293
4560293
4707313
4821017
4842956
4860902
4891032
4918748
4922567
4926506
5125578
5180331
5302878
5386601
5544859
5634908
5679938
5679938
5679938
5683098
5708206
5708206
5722142
5722142
5779804
6000741
6000741
6085020
6145716
6158479
6242952
6242952
6324940
6338386
6338386
6518811
6588818
6591824
6610460
6610460
6610460
6655263
6800640
6884107
6982291
7079833
7234285
7234285
7251581
7284844
7284844
7284844
7284844
7309258
7309258
7423611
7480196
7531700
7564885
7689281
7719716
7731315
7732089
7732089
7754134
7869986
7882556
7891161
7891161
7901590
7901590
7901590
7901590
7967752
7967752
8012028
8150786
8231866
8399009
8435856
8506469
8838428
8839333
8839333
8955947
8981088
9224530
9243738
9391429
9391429
9513550
9574993
9601149
9687004
9691399
9739796
9739796


In [796]:
pd.to_datetime(temp.loc[0,'分裂日期'])+datetime.timedelta(days=1)

Timestamp('2020-08-24 00:00:00')

In [797]:
temp.loc[0,'tdm_date']

'2020-08-24'

In [798]:
temp

,patient_id,case_no,drug_name,amount,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc,...,氯离子,肌酸激酶,钠离子,钾离子,脂肪酶,总钙,无机磷,胱抑素-C,镁离子,葡萄糖
0,6865481,9910592,丙戊酸钠糖浆[100ml:5g*1瓶],1,100ml*1瓶,0.25,1,2020-08-18 08:54:42,2020-08-27 08:08:00,停止,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [799]:
temp_other_1

,test_record_id,patient_id,case_no,test_date,clinical_diagnosis,sample_type,sample_time,project_name,test_result,result_unit,is_normal,refer_scope,synonym
0,800008499654||A035||1,6865481,9910592,2020-08-24,双相情感障碍，目前为伴有精神病性症状的躁狂发作,尿液,NaT,酸碱浓度,7.500,None,N,4.5-8.0,PH
1,800008678249||A083||1,6865481,9910592,2020-08-24,双相情感障碍，目前为伴有精神病性症状的躁狂发作,静脉血,NaT,嗜酸性粒细胞总数,0.250,×10~9/L,N,0.02-0.52,EOS
2,800008678249||A083||1,6865481,9910592,2020-08-24,双相情感障碍，目前为伴有精神病性症状的躁狂发作,静脉血,NaT,嗜碱性粒细胞总数,0.020,×10~9/L,N,0.00-0.06,BASO
3,800008678249||A083||1,6865481,9910592,2020-08-24,双相情感障碍，目前为伴有精神病性症状的躁狂发作,静脉血,NaT,淋巴细胞百分数,33.900,%,N,20.0-50.0,LYM%
4,800008678249||A083||1,6865481,9910592,2020-08-24,双相情感障碍，目前为伴有精神病性症状的躁狂发作,静脉血,NaT,中性粒细胞百分数,56.200,%,N,40.0-75.0,NEU%
5,800008678249||A083||1,6865481,9910592,2020-08-24,双相情感障碍，目前为伴有精神病性症状的躁狂发作,静脉血,NaT,单核细胞百分数,6.500,%,N,3.0-10.0,MONO%
6,800008678249||A083||1,6865481,9910592,2020-08-24,双相情感障碍，目前为伴有精神病性症状的躁狂发作,静脉血,NaT,嗜酸性粒细胞百分数,3.100,%,N,0.4-8.0,EOS%
7,800008678249||A083||1,6865481,9910592,2020-08-24,双相情感障碍，目前为伴有精神病性症状的躁狂发作,静脉血,NaT,嗜碱性粒细胞百分数,0.300,%,N,0.0-1.0,BASO%
8,800008678249||A083||1,6865481,9910592,2020-08-24,双相情感障碍，目前为伴有精神病性症状的躁狂发作,静脉血,NaT,红细胞计数,5.410,×10~12/L,N,4.30-5.80,RBC
9,800008678249||A083||1,6865481,9910592,2020-08-24,双相情感障碍，目前为伴有精神病性症状的躁狂发作,静脉血,NaT,单核细胞总数,0.520,×10~9/L,N,0.10-0.60,MONO


In [800]:
print(df_VPA_test_other.shape)

(127, 97)


In [801]:
df_VPA_test_other

,patient_id,case_no,drug_name,amount,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc,...,氯离子,肌酸激酶,钠离子,钾离子,脂肪酶,总钙,无机磷,胱抑素-C,镁离子,葡萄糖
0,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2020-08-24 10:37:31,2020-09-04 11:38:00,停止,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,547307,10005574,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2020-08-16 16:41:46,2020-08-31 11:15:00,停止,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6949939,10019847,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2020-08-17 21:10:00,2020-09-03 08:26:00,停止,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,369329,10028911,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2020-08-18 16:35:34,2020-09-02 08:34:00,停止,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8413999,10171606,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2020-09-04 10:50:11,2020-09-15 09:09:00,停止,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,8029177,9739796,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2020-07-20 10:49:58,2020-07-27 11:02:39,停止,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123,8029177,9739796,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-07-27 11:07:39,2020-08-08 08:13:00,停止,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124,8325164,9812242,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.75,1,2020-08-03 10:43:56,2020-08-14 08:16:00,停止,...,102.7,NaN,137.0,3.92,NaN,NaN,NaN,NaN,NaN,NaN
125,8325164,9812242,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.75,1,2020-08-03 10:43:56,2020-08-14 08:16:00,停止,...,102.7,NaN,137.0,3.92,NaN,NaN,NaN,NaN,NaN,NaN


In [802]:
# 合并其他检测
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_7.3_合并其他检测.xlsx')
df_VPA_test_other.to_excel(writer)
writer.save()

## 删除缺失率超过50%的列

In [803]:
# 删除缺失超过50%的检测
for i in df_VPA_tdm.columns:
    x=df_VPA_tdm[i].isnull().sum()
    per_x = x/df_VPA_tdm.shape[0]
    if per_x >= 0.5:
        print(i)
        del df_VPA_tdm[i]

In [804]:
print(df_VPA_tdm.shape)

(127, 26)


In [805]:
# 保存删除缺失率后的数据
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_7.4_删除缺失超过50%的列.xlsx')
df_VPA_tdm.to_excel(writer)
writer.save()

## 人工删除无意义列

In [ ]:
del df_VPA_group['未分类管型']
del df_VPA_group['比重']
del df_VPA_group['透明管型']
del df_VPA_group['粘液']
del df_VPA_group['未分类结晶']
del df_VPA_group['细胞比积']
del df_VPA_group['钙(干式)']
del df_VPA_group['非鳞状上皮细胞']
del df_VPA_group['鳞状上皮细胞']

In [ ]:
print(df_VPA_group.shape)

In [ ]:
# 保存删除缺失率后的数据
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_11.4_人工删除无意义列.xlsx')
df_VPA_group.to_excel(writer)
writer.save()